# Revisión de datos de los archivos csv de Open Contracting Partnership. Data Set del 2021.

Usando el buscador de OpenContracting se buscaron datos sobre Chile (buscador: https://data.open-contracting.org/es/search/). Se decidió descargarlos archivos de los años 2018 y 2021, para tener una muestra de datos que contenga información sobre licitaciones en un rango de tiempo cercano a estas fechas.

In [1]:
import pandas as pd
import numpy as np
import datetime
from os import path

## Revision de main.csv

In [2]:
camino = path.join("Chile","2021","main.csv")
df_main = pd.read_csv(camino, sep=",",low_memory=False)

In [3]:
df_respaldo = df_main

In [4]:
df_main.head()

,_link,id,tag,date,ocid,language,initiationType,tender_id,tender_procurementMethodDetails,tender_title,...,buyer_name,planning_budget_id,planning_budget_description,planning_budget_amount_amount,planning_budget_amount_currency,tender_techniques_hasFrameworkAgreement,tender_techniques_frameworkAgreement_method,tender_contractPeriod_endDate,tender_contractPeriod_startDate,tender_contractPeriod_durationInDays
0,id-0.0,ocds-70d2nz-1057501-322-LQ21-2021-12-22T15:00:11Z,compiled,2021-12-22T15:00:11Z,ocds-70d2nz-1057501-322-LQ21,es,tender,1057501-322-LQ21,Licitación Pública entre a 2000 y 5000 UTM (LQ),CONVENIO INSUMOS PABELLON CENTRAL II,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id-0.1,ocds-70d2nz-1039043-27-LE21-2021-12-29T17:32:40Z,compiled,2021-12-29T17:32:40Z,ocds-70d2nz-1039043-27-LE21,es,tender,1039043-27-LE21,Licitación Pública Entre 100 y 1000 UTM (LE),LIC. SERVICIOS MANTENCION DE EDIFICIOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,id-0.2,ocds-70d2nz-2281-300-L121-2021-12-28T17:15:48Z,compiled,2021-12-28T17:15:48Z,ocds-70d2nz-2281-300-L121,es,tender,2281-300-L121,Licitación Pública Menor a 100 UTM (L1),ADQUISICION DE MATERIALES DE OFICINAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,id-0.3,ocds-70d2nz-968521-14-LE21-2021-12-27T11:47:53Z,compiled,2021-12-27T11:47:53Z,ocds-70d2nz-968521-14-LE21,es,tender,968521-14-LE21,Licitación Pública Entre 100 y 1000 UTM (LE),Antisépticos y desinfectantes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,compiled,2021-12-27T17:31:16Z,ocds-70d2nz-1079639-135-L121,es,tender,1079639-135-L121,Licitación Pública Menor a 100 UTM (L1),CIERRE PERIMETRAL DE 04 VIVIENDAS FISCALES DE ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134701 entries, 0 to 134700
Data columns (total 39 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   _link                                        134701 non-null  object 
 1   id                                           134701 non-null  object 
 2   tag                                          134701 non-null  object 
 3   date                                         134701 non-null  object 
 4   ocid                                         134701 non-null  object 
 5   language                                     134701 non-null  object 
 6   initiationType                               134701 non-null  object 
 7   tender_id                                    134701 non-null  object 
 8   tender_procurementMethodDetails              134701 non-null  object 
 9   tender_title                                 134701 non-nul

In [6]:
df_main.columns

Index(['_link', 'id', 'tag', 'date', 'ocid', 'language', 'initiationType',
       'tender_id', 'tender_procurementMethodDetails', 'tender_title',
       'tender_status', 'tender_procurementMethod', 'tender_description',
       'tender_hasEnquiries', 'tender_awardPeriod_endDate',
       'tender_awardPeriod_startDate', 'tender_awardPeriod_durationInDays',
       'tender_tenderPeriod_endDate', 'tender_tenderPeriod_startDate',
       'tender_tenderPeriod_durationInDays', 'tender_enquiryPeriod_endDate',
       'tender_enquiryPeriod_startDate', 'tender_enquiryPeriod_durationInDays',
       'tender_procuringEntity_id', 'tender_procuringEntity_name',
       'tender_value_amount', 'tender_value_unitOfAccount',
       'tender_value_currency', 'buyer_id', 'buyer_name', 'planning_budget_id',
       'planning_budget_description', 'planning_budget_amount_amount',
       'planning_budget_amount_currency',
       'tender_techniques_hasFrameworkAgreement',
       'tender_techniques_frameworkAgreement_m

In [7]:
df_main.drop(columns=['tag', 'date', 'ocid', 'language', 'tender_title', 'tender_description', 'planning_budget_id','planning_budget_description', 'planning_budget_amount_amount','planning_budget_amount_currency','tender_techniques_hasFrameworkAgreement',
       'tender_techniques_frameworkAgreement_method','tender_contractPeriod_endDate', 'tender_contractPeriod_startDate','tender_contractPeriod_durationInDays'], inplace=True)

In [8]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134701 entries, 0 to 134700
Data columns (total 24 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   _link                                134701 non-null  object 
 1   id                                   134701 non-null  object 
 2   initiationType                       134701 non-null  object 
 3   tender_id                            134701 non-null  object 
 4   tender_procurementMethodDetails      134701 non-null  object 
 5   tender_status                        134701 non-null  object 
 6   tender_procurementMethod             134701 non-null  object 
 7   tender_hasEnquiries                  134701 non-null  bool   
 8   tender_awardPeriod_endDate           131901 non-null  object 
 9   tender_awardPeriod_startDate         134688 non-null  object 
 10  tender_awardPeriod_durationInDays    130283 non-null  float64
 11  tender_tender

In [9]:
df_main["tender_procurementMethodDetails"].unique()

array(['Licitación Pública entre a 2000 y 5000 UTM (LQ)',
       'Licitación Pública Entre 100 y 1000 UTM (LE)',
       'Licitación Pública Menor a 100 UTM (L1)',
       'Licitación Pública Mayor 1000 UTM (LP)',
       'Licitación Pública Mayor a 5000 (LR)',
       'Licitación Privada entre 100 y 1000 UTM.',
       'Licitación Pública MOP (O1)',
       'Licitación Privada Menor a 100 UTM.',
       'Licitación Privada MOP (O2)',
       'Licitación Pública Servicios personales especializados (LS)',
       'Licitación Privada Mayor a 1000 UTM',
       'Licitación Privada entre a 2000 y 5000 UTM (H2)',
       'Licitación Privada Mayor a 5000 (I2)'], dtype=object)

Se utilizan la pagina <<https://api.mercadopublico.cl/modules/Licitacion.aspx>> para crear el diccionario.

In [10]:

tipos_licitaciones ={'Licitación Pública Menor a 100 UTM (L1)': "L1",
       'Licitación Pública Entre 100 y 1000 UTM (LE)': "LE",
       'Licitación Pública Mayor 1000 UTM (LP)': "LP",
       'Licitación Pública entre a 2000 y 5000 UTM (LQ)':"LQ",
       'Licitación Pública Mayor a 5000 (LR)' :"LR",
       'Licitación Privada entre a 2000 y 5000 UTM (H2)':"H2",
       'Licitación Pública Servicios personales especializados (LS)':"LS",
       "Licitación Privada por Licitación Pública anterior sin oferentes (A1)" : "A1",
       "Licitación Privada por Remanente de Contrato anterior (B1)": "B1",
       "Licitación Privada por Convenios con Personas Jurídicas Extranjeras fuera del Territorio Nacional (E1)": "E1",
       "Licitación Privada por Servicios de Naturaleza Confidencial (F1)": "F1",
       "Licitación Privada por otras causales, excluidas de la ley de Compras (J1)": "J1",
       "Licitación Privada entre 100 y 1000 UTM" :"CO", "Licitación Privada Mayor a 1000 UTM": "B2",
       "Trato Directo por Producto de Licitación Privada anterior sin oferentes o desierta": "A2",
       "Trato Directo por Proveedor Único (D1)": "D1","Licitación Privada Menor a 100 UTM" : "E2", "Trato Directo (Cotización) (C2)" : "C2",
       "Compra Directa (Orden de compra) (C1)" : "C1", "Trato Directo (Cotización) (F2)" :"F2", "Compra Directa (Orden de compra) (F3)" : "F3",
       "Directo (Cotización) (G2)" : "G2", "Compra Directa (Orden de compra) (G1)" : "G1",
       "Orden de Compra menor a 3 UTM (R1)" : "R1", "Orden de Compra sin Resolución (CA)" : "CA", 
       "Orden de Compra proveniente de adquisición sin emisión automática de OC (SE)" : "SE"}


df_main["tender_procurementMethodDetails"] = df_main["tender_procurementMethodDetails"].map(tipos_licitaciones)
df_main["tender_procurementMethodDetails"] = df_main["tender_procurementMethodDetails"].astype("category")

In [11]:
df_main["buyer_id"] = df_main["tender_procuringEntity_id"]
df_main['buyer_id'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 134701 entries, 0 to 134700
Series name: buyer_id
Non-Null Count   Dtype 
--------------   ----- 
134701 non-null  object
dtypes: object(1)
memory usage: 1.0+ MB


In [12]:
def dejar_rut(texto:str) -> str:
    return int(texto.split("-")[2])

df_main["buyer_id"] = df_main["buyer_id"].apply(dejar_rut)

In [13]:
df_main.head()

,_link,id,initiationType,tender_id,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,tender_enquiryPeriod_endDate,tender_enquiryPeriod_startDate,tender_enquiryPeriod_durationInDays,tender_procuringEntity_id,tender_procuringEntity_name,tender_value_amount,tender_value_unitOfAccount,tender_value_currency,buyer_id,buyer_name
0,id-0.0,ocds-70d2nz-1057501-322-LQ21-2021-12-22T15:00:11Z,tender,1057501-322-LQ21,LQ,active,open,True,2022-02-15T17:00:00Z,2021-12-22T15:01:00Z,...,2021-12-09T15:00:00Z,2021-12-07T17:21:00Z,2.0,CL-MP-1057501,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Bien...,NaN,NaN,NaN,1057501,NaN
1,id-0.1,ocds-70d2nz-1039043-27-LE21-2021-12-29T17:32:40Z,tender,1039043-27-LE21,LE,active,open,False,2022-01-13T18:00:00Z,2022-01-07T15:31:00Z,...,2021-12-31T16:00:00Z,2021-12-29T18:01:00Z,2.0,CL-MP-1039043,INSTITUTO DE DESARROLLO AGROPECUARIO | INSTITU...,999.0,UTM,NaN,1039043,NaN
2,id-0.2,ocds-70d2nz-2281-300-L121-2021-12-28T17:15:48Z,tender,2281-300-L121,L1,active,open,False,2022-01-11T20:51:00Z,2022-01-04T16:55:00Z,...,2021-12-30T20:51:00Z,2021-12-28T17:15:47Z,2.0,CL-MP-3275,I MUNICIPALIDAD VALDIVIA | UNIDAD ADQUISICIONE...,2800000.0,NaN,CLP,3275,NaN
3,id-0.3,ocds-70d2nz-968521-14-LE21-2021-12-27T11:47:53Z,tender,968521-14-LE21,LE,active,open,True,2022-01-14T10:07:00Z,2021-12-27T10:07:00Z,...,2021-12-20T20:56:00Z,2021-12-16T20:55:00Z,4.0,CL-MP-968521,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIE...,NaN,NaN,NaN,968521,NaN
4,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22T18:00:00Z,2021-12-07T15:10:00Z,...,2021-12-04T18:00:00Z,2021-12-02T19:30:51Z,2.0,CL-MP-1079639,DIRECCION DE LOGISTICA DE CARABINEROS | SECCIÓ...,5000000.0,NaN,CLP,1079639,NaN


In [14]:
df_main.drop(columns=['tender_procuringEntity_id'], inplace=True)

In [15]:
df_main.loc[df_main['tender_enquiryPeriod_startDate'] == "3"]

,_link,id,initiationType,tender_id,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,tender_tenderPeriod_durationInDays,tender_enquiryPeriod_endDate,tender_enquiryPeriod_startDate,tender_enquiryPeriod_durationInDays,tender_procuringEntity_name,tender_value_amount,tender_value_unitOfAccount,tender_value_currency,buyer_id,buyer_name


In [16]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134701 entries, 0 to 134700
Data columns (total 23 columns):
 #   Column                               Non-Null Count   Dtype   
---  ------                               --------------   -----   
 0   _link                                134701 non-null  object  
 1   id                                   134701 non-null  object  
 2   initiationType                       134701 non-null  object  
 3   tender_id                            134701 non-null  object  
 4   tender_procurementMethodDetails      130667 non-null  category
 5   tender_status                        134701 non-null  object  
 6   tender_procurementMethod             134701 non-null  object  
 7   tender_hasEnquiries                  134701 non-null  bool    
 8   tender_awardPeriod_endDate           131901 non-null  object  
 9   tender_awardPeriod_startDate         134688 non-null  object  
 10  tender_awardPeriod_durationInDays    130283 non-null  float64 
 11  

### Transformación de datos.

In [17]:
df_main["tender_enquiryPeriod_startDate"] = pd.to_datetime(df_main['tender_enquiryPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main["tender_enquiryPeriod_endDate"] = pd.to_datetime(df_main['tender_enquiryPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main['tender_tenderPeriod_startDate'] = pd.to_datetime(df_main['tender_tenderPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main["tender_tenderPeriod_endDate"] = pd.to_datetime(df_main['tender_tenderPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main['tender_awardPeriod_startDate'] = pd.to_datetime(df_main['tender_awardPeriod_startDate'], format='%Y-%m-%dT%H:%M:%SZ')

Hay un error en el formato de una fila con contenido <2999-03-15T12:17:00Z>, se eliminará.

In [18]:
df_main.drop(df_main[df_main["tender_awardPeriod_endDate"] == "2999-03-15T12:17:00Z"].index, axis=0, inplace=True)
df_main.drop(df_main[df_main["tender_awardPeriod_endDate"] == "2999-05-31T17:26:00Z"].index, axis=0, inplace=True)
df_main.drop(df_main[df_main["tender_awardPeriod_endDate"] == "2999-05-31T15:11:00Z"].index, axis=0, inplace=True)

In [19]:
df_main['tender_awardPeriod_endDate'] = pd.to_datetime(df_main['tender_awardPeriod_endDate'], format='%Y-%m-%dT%H:%M:%SZ')
df_main['tender_enquiryPeriod_durationInDays'] = df_main['tender_enquiryPeriod_endDate'] - df_main['tender_enquiryPeriod_startDate'] 
df_main['tender_awardPeriod_durationInDays'] = df_main['tender_awardPeriod_endDate'] - df_main['tender_awardPeriod_startDate']
df_main['tender_tenderPeriod_durationInDays'] = df_main['tender_tenderPeriod_endDate'] - df_main['tender_tenderPeriod_startDate']

# Filtrado por fecha
Así como en el notebook del 2018, filtraremos las fechas, de forma que queden solo entre (2020-09-01) y (2021-12-31).

In [20]:
fecha_filtro = '2021-01-01'
df_filtrado = df_main.loc[df_main['tender_tenderPeriod_startDate'] >= fecha_filtro]

fecha_filtro = '2021-12-31'
df_filtrado = df_filtrado.loc[df_filtrado['tender_awardPeriod_endDate'] <= fecha_filtro]

In [21]:
df_filtrado.reset_index(drop=True, inplace=True)
df_filtrado.rename(columns={"tender_value_amount":"estimated_cost", "tender_value_currency": "estimated_cost_currency", "tender_value_unitOfAccount": "estimated_cost_unitOfAccount"}, inplace=True )
df_filtrado["estimated_cost_currency"] = df_filtrado["estimated_cost_currency"].replace(np.nan, "No")

In [22]:
def crear_unit_value_monetary_unit(indice:int, comparador="No") -> str:
    if df_filtrado.iloc[indice]["estimated_cost_currency"] == comparador:
        return df_filtrado.iloc[indice]["estimated_cost_unitOfAccount"]
    return df_filtrado.iloc[indice]["estimated_cost_currency"]

df_filtrado["estimated_cost_monetary_unit"] = df_filtrado.index.map(crear_unit_value_monetary_unit)

In [23]:
df_filtrado["estimated_cost_monetary_unit"].unique()

array(['CLP', nan, 'CLF', 'USD', 'UTM', 'EUR'], dtype=object)

In [24]:
df_filtrado.drop(columns=["estimated_cost_currency","estimated_cost_unitOfAccount"],inplace=True)
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116359 entries, 0 to 116358
Data columns (total 22 columns):
 #   Column                               Non-Null Count   Dtype          
---  ------                               --------------   -----          
 0   _link                                116359 non-null  object         
 1   id                                   116359 non-null  object         
 2   initiationType                       116359 non-null  object         
 3   tender_id                            116359 non-null  object         
 4   tender_procurementMethodDetails      115246 non-null  category       
 5   tender_status                        116359 non-null  object         
 6   tender_procurementMethod             116359 non-null  object         
 7   tender_hasEnquiries                  116359 non-null  bool           
 8   tender_awardPeriod_endDate           116359 non-null  datetime64[ns] 
 9   tender_awardPeriod_startDate         116359 non-null  datet

## Revision awards.csv

In [25]:
camino = path.join("Chile","2018","awards.csv")
df_awards = pd.read_csv(camino, sep=",")

In [26]:
df_awards.head(8)

,_link,_link_main,id,date,description,title,status,value_amount,value_currency,value_unitOfAccount
0,id-0.0.awards.0,id-0.0,8270925,2017-12-23T13:23:24Z,DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES CO...,DIDECO - NAVIDAD COMUNAL - JUEGOS INFLABLES,active,690000.0,CLP,NaN
1,id-0.1.awards.0,id-0.1,8274205,NaN,CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METAL...,CONFECCIÓN E INSTALACIÓN DE PROTECCIONES METÁL...,unsuccessful,NaN,NaN,NaN
2,id-0.2.awards.0,id-0.2,8275841,2018-02-02T11:16:54Z,sm80 sp3565 servicios generales,Reparacion de mamparas,active,870000.0,CLP,NaN
3,id-0.3.awards.0,id-0.3,8254526,2017-12-26T18:19:33Z,VISITA A HUERTO ORGANICO DE BERRIES HUBICADO E...,VISTA A HUERTO ORGANICO DE BERRIES HUBICADO EN...,unsuccessful,NaN,NaN,NaN
4,id-0.4.awards.0,id-0.4,8253535,2017-12-21T18:17:23Z,CAPACITACIÓN PARA DOCENTES DE LA ESC. F-929 - ...,CAPACITACIÓN PARA DOCENTES DE LA ESC. F-929 - ...,active,6000000.0,CLP,NaN
5,id-0.5.awards.0,id-0.5,8277247,2018-01-16T10:56:54Z,Adquisicion requerida por el Sr. Juan Pablo Re...,Adquisición programa rehabilitación integral e...,active,347043.0,CLP,NaN
6,id-0.6.awards.0,id-0.6,8274542,2017-12-28T21:54:24Z,SE REQUIERE LA ADQUISICION DE REACTIVOS PARA L...,REACTIVOS PARA LABORATORIO,active,12219604.0,CLP,NaN
7,id-0.7.awards.0,id-0.7,8271070,2017-12-29T11:54:57Z,PROGRAMA ATENCIÓN INTEGRAL FAMILIAR 24 HORAS,TERMO-VENTILADOR Y ESTUFA A GAS (N°1017) COM.S...,active,191241.0,CLP,NaN


In [27]:
df_no_nan_un_ac = df_awards.dropna(subset=["value_unitOfAccount"])[["value_currency", "value_unitOfAccount", "status"]]
df_unit_of_account = df_no_nan_un_ac.loc[df_no_nan_un_ac["value_currency"].isna()]

In [28]:
df_unit_of_account["value_unitOfAccount"].describe()

count      83
unique      1
top       UTM
freq       83
Name: value_unitOfAccount, dtype: object

In [29]:
df_no_nan_un_ac["value_unitOfAccount"].describe()

count      84
unique      2
top       UTM
freq       83
Name: value_unitOfAccount, dtype: object

In [30]:
df_awards["value_currency"] = df_awards["value_currency"].replace(np.nan, "No")

In [31]:
def crear_unit_value_monetary_unit(indice:int, comparador="No") -> str:
    if df_awards.iloc[indice]["value_currency"] == comparador:
        return df_awards.iloc[indice]["value_unitOfAccount"]
    return df_awards.iloc[indice]["value_currency"]

df_awards["value_monetary_unit"] = df_awards.index.map(crear_unit_value_monetary_unit)

In [32]:
df_awards["value_monetary_unit"].unique()

array(['CLP', nan, 'CLF', 'USD', 'UTM', 'EUR'], dtype=object)

In [33]:
df_awards.drop(columns=['id', 'description', 'title', "value_unitOfAccount", "value_currency","value_unitOfAccount"],inplace=True)

In [34]:
df_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240021 entries, 0 to 240020
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _link                240021 non-null  object 
 1   _link_main           240021 non-null  object 
 2   date                 210324 non-null  object 
 3   status               240021 non-null  object 
 4   value_amount         177451 non-null  float64
 5   value_monetary_unit  177451 non-null  object 
dtypes: float64(1), object(5)
memory usage: 11.0+ MB


## Revisión de awards_items.csv

In [35]:
camino = path.join("Chile","2021","awards_items.csv") 
df_awards_items = pd.read_csv(camino, sep=",")

In [36]:
df_awards_items.columns

Index(['_link', '_link_awards', '_link_main', 'id', 'description', 'quantity',
       'unit_name', 'unit_value_amount', 'unit_value_currency',
       'classification_id', 'classification_uri', 'classification_scheme',
       'unit_value_unitOfAccount'],
      dtype='object')

In [37]:
df_awards_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359039 entries, 0 to 359038
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   _link                     359039 non-null  object 
 1   _link_awards              359039 non-null  object 
 2   _link_main                359039 non-null  object 
 3   id                        359039 non-null  int64  
 4   description               357622 non-null  object 
 5   quantity                  359039 non-null  float64
 6   unit_name                 359039 non-null  object 
 7   unit_value_amount         359039 non-null  float64
 8   unit_value_currency       358833 non-null  object 
 9   classification_id         359039 non-null  int64  
 10  classification_uri        359039 non-null  object 
 11  classification_scheme     359039 non-null  object 
 12  unit_value_unitOfAccount  206 non-null     object 
dtypes: float64(2), int64(2), object(9)
memory us

In [38]:
df_awards_items["unit_value_unitOfAccount"].describe()

count     206
unique      1
top       UTM
freq      206
Name: unit_value_unitOfAccount, dtype: object

In [39]:
df_awards_items.loc[df_awards_items["unit_value_currency"].isna()]["unit_value_unitOfAccount"].describe()

count     206
unique      1
top       UTM
freq      206
Name: unit_value_unitOfAccount, dtype: object

Podemos observar que para los valores de <unit_value_currency> que son NaN existe un dato en <unit_value_unitOfAccount>, por lo que crearemos una nueva columna llamada <unit_value_monetary_unit> que tenga los valores de ambas columna según correspondan.

In [40]:
df_awards_items["unit_value_currency"] = df_awards_items["unit_value_currency"].replace(np.nan, "No")

In [41]:
def crear_unit_value_monetary_unit(indice:int, comparador="No") -> str:
    if df_awards_items.iloc[indice]["unit_value_currency"] == comparador:
        return df_awards_items.iloc[indice]["unit_value_unitOfAccount"]
    return df_awards_items.iloc[indice]["unit_value_currency"]

df_awards_items["unit_value_monetary_unit"] = df_awards_items.index.map(crear_unit_value_monetary_unit)

In [42]:
df_awards_items["unit_value_monetary_unit"].unique()

array(['CLP', 'USD', 'CLF', 'UTM', 'EUR'], dtype=object)

In [43]:
df_awards_items.drop(columns=['classification_uri', 'classification_scheme','unit_value_unitOfAccount', "unit_value_currency", "description"], inplace=True)

## Revisión de awards_suppliers.csv

In [44]:
camino = path.join("Chile","2021","awards_suppliers.csv")
df_awards_supplier = pd.read_csv(camino, sep=",")

In [45]:
df_awards_supplier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134056 entries, 0 to 134055
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   _link         134056 non-null  object
 1   _link_awards  134056 non-null  object
 2   _link_main    134056 non-null  object
 3   id            134056 non-null  object
 4   name          134056 non-null  object
dtypes: object(5)
memory usage: 5.1+ MB


In [46]:
df_awards_supplier.head()

,_link,_link_awards,_link_main,id,name
0,id-0.9.awards.0.suppliers.0,id-0.9.awards.0,id-0.9,CL-MP-106395,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.
1,id-0.9.awards.0.suppliers.1,id-0.9.awards.0,id-0.9,CL-MP-24938,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA
2,id-0.9.awards.0.suppliers.2,id-0.9.awards.0,id-0.9,CL-MP-44934,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...
3,id-0.9.awards.0.suppliers.3,id-0.9.awards.0,id-0.9,CL-MP-54079,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.
4,id-0.9.awards.0.suppliers.4,id-0.9.awards.0,id-0.9,CL-MP-747806,Indopharma S.A. | Indopharma S.A.


In [47]:
def dejar_rut(texto:str) -> str:
    return int(texto.split("-")[2])

In [48]:
df_awards_supplier["id"] = df_awards_supplier["id"].apply(dejar_rut)

In [49]:
df_awards_supplier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134056 entries, 0 to 134055
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   _link         134056 non-null  object
 1   _link_awards  134056 non-null  object
 2   _link_main    134056 non-null  object
 3   id            134056 non-null  int64 
 4   name          134056 non-null  object
dtypes: int64(1), object(4)
memory usage: 5.1+ MB


## Revisión de awards_documents.csv

In [50]:
camino = path.join("Chile","2021", "awards_documents.csv")
df_award_documents = pd.read_csv(camino, sep=",")
df_award_documents.head()

,_link,_link_awards,_link_main,id,url,title,format,language,description,documentType
0,id-0.9.awards.0.documents.0,id-0.9.awards.0,id-0.9,1,https://www.mercadopublico.cl/Procurement/Modu...,Página documentos del proceso de contratación,html,es,Todos los documentos relacionados al proceso d...,x_procurementDocuments
1,id-0.11.awards.0.documents.0,id-0.11.awards.0,id-0.11,1,https://www.mercadopublico.cl/Procurement/Modu...,Página documentos del proceso de contratación,html,es,Todos los documentos relacionados al proceso d...,x_procurementDocuments
2,id-0.14.awards.0.documents.0,id-0.14.awards.0,id-0.14,1,https://www.mercadopublico.cl/Procurement/Modu...,Página documentos del proceso de contratación,html,es,Todos los documentos relacionados al proceso d...,x_procurementDocuments
3,id-0.15.awards.0.documents.0,id-0.15.awards.0,id-0.15,1,https://www.mercadopublico.cl/Procurement/Modu...,Página documentos del proceso de contratación,html,es,Todos los documentos relacionados al proceso d...,x_procurementDocuments
4,id-0.19.awards.0.documents.0,id-0.19.awards.0,id-0.19,1,https://www.mercadopublico.cl/Procurement/Modu...,Página documentos del proceso de contratación,html,es,Todos los documentos relacionados al proceso d...,x_procurementDocuments


In [51]:
df_award_documents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85675 entries, 0 to 85674
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _link         85675 non-null  object
 1   _link_awards  85675 non-null  object
 2   _link_main    85675 non-null  object
 3   id            85675 non-null  int64 
 4   url           85675 non-null  object
 5   title         85675 non-null  object
 6   format        85675 non-null  object
 7   language      85675 non-null  object
 8   description   85675 non-null  object
 9   documentType  85675 non-null  object
dtypes: int64(1), object(9)
memory usage: 6.5+ MB


No se utilizará este DataFrame, puesto que no contiene información relevante para lo que se busca resolver.

## Revisión de parties.csv

In [52]:
camino = path.join("Chile","2021", "parties.csv")
df_parties = pd.read_csv(camino, sep=",")
df_parties.head()

,_link,_link_main,id,name,roles,address_region,address_countryName,address_streetAddress,identifier_id,identifier_scheme,identifier_legalName,contactPoint_name,contactPoint_email,contactPoint_telephone,contactPoint_faxNumber
0,id-0.0.parties.0,id-0.0,CL-MP-1057501,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Bien...,"procuringEntity,buyer",Región Metropolitana de Santiago,Chile,"Avenida Melchor Concha y Toro 3459, Puente Alto",616085026,CL-RUT,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO,JORGE RUBIO,jorge.rubio@ssmso.cl,56-02-25764947,NaN
1,id-0.1.parties.0,id-0.1,CL-MP-1039043,INSTITUTO DE DESARROLLO AGROPECUARIO | INSTITU...,"procuringEntity,buyer",Región del Ñuble,Chile,"Claudio Arrau 738, Chillán",613070001,CL-RUT,INSTITUTO DE DESARROLLO AGROPECUARIO,Sergio Sepulveda Brito,ssepulvedab@indap.cl,56-42-2386393,NaN
2,id-0.2.parties.0,id-0.2,CL-MP-3275,I MUNICIPALIDAD VALDIVIA | UNIDAD ADQUISICIONE...,"procuringEntity,buyer",Región de Los Ríos,Chile,"BAQUEDANO, 1165",692001001,CL-RUT,I MUNICIPALIDAD VALDIVIA,LUIS MARCOS GALAZ SANZANA,mgalaz@munivaldivia.cl,56-63-2288629-8629,NaN
3,id-0.3.parties.0,id-0.3,CL-MP-968521,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIE...,"procuringEntity,buyer",Región del Maule,Chile,RIQUELME 1131 - Región del Maule - San Javier,616069128,CL-RUT,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIER,Carol Andrea Soto Troncoso,carol.soto.t@gmail.com,56-73-2566358,NaN
4,id-0.3.parties.1,id-0.3,CL-MP-1011396,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD,tenderer,Región Metropolitana de Santiago,Chile,Suecia 0142 202,768357935,CL-RUT,BEFORE PUBLICIDAD SPA,Karla Lugo,karlalugo17@gmail.com,56-2-33399859,NaN


In [53]:
df_parties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606887 entries, 0 to 606886
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   _link                   606887 non-null  object
 1   _link_main              606887 non-null  object
 2   id                      606887 non-null  object
 3   name                    606887 non-null  object
 4   roles                   606887 non-null  object
 5   address_region          598288 non-null  object
 6   address_countryName     554278 non-null  object
 7   address_streetAddress   600390 non-null  object
 8   identifier_id           606430 non-null  object
 9   identifier_scheme       606434 non-null  object
 10  identifier_legalName    606327 non-null  object
 11  contactPoint_name       606887 non-null  object
 12  contactPoint_email      606878 non-null  object
 13  contactPoint_telephone  584777 non-null  object
 14  contactPoint_faxNumber  91647 non-nu

In [54]:
df_parties.drop(columns=['address_streetAddress', 'identifier_scheme','identifier_legalName', 'contactPoint_name', 'contactPoint_email','contactPoint_telephone', 'contactPoint_faxNumber'], inplace=True)

In [55]:
df_parties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606887 entries, 0 to 606886
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   _link                606887 non-null  object
 1   _link_main           606887 non-null  object
 2   id                   606887 non-null  object
 3   name                 606887 non-null  object
 4   roles                606887 non-null  object
 5   address_region       598288 non-null  object
 6   address_countryName  554278 non-null  object
 7   identifier_id        606430 non-null  object
dtypes: object(8)
memory usage: 37.0+ MB


Se eliminan estas columnas porque ofrecen información que no es de interes, ni se puede utilizar para obtener información en alguna de las paginas relacionadas al dataset.

In [56]:
df_parties["roles"].unique()

array(['procuringEntity,buyer', 'tenderer', 'supplier,tenderer',
       'procuringEntity'], dtype=object)

In [57]:
def separar_roles(texto:str) -> list:
    if "," in texto:
        return texto.split(",")
    else:
        return [texto, np.nan]
    
df_parties["roles"] = df_parties["roles"].apply(separar_roles)

In [58]:
df_parties["rol 1"] = df_parties.index.map(lambda x: df_parties.loc[x]["roles"][0])

In [59]:
df_parties["rol 2"] = df_parties.index.map(lambda x: df_parties.loc[x]["roles"][1])

In [60]:
df_parties.head()

,_link,_link_main,id,name,roles,address_region,address_countryName,identifier_id,rol 1,rol 2
0,id-0.0.parties.0,id-0.0,CL-MP-1057501,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Bien...,"[procuringEntity, buyer]",Región Metropolitana de Santiago,Chile,616085026,procuringEntity,buyer
1,id-0.1.parties.0,id-0.1,CL-MP-1039043,INSTITUTO DE DESARROLLO AGROPECUARIO | INSTITU...,"[procuringEntity, buyer]",Región del Ñuble,Chile,613070001,procuringEntity,buyer
2,id-0.2.parties.0,id-0.2,CL-MP-3275,I MUNICIPALIDAD VALDIVIA | UNIDAD ADQUISICIONE...,"[procuringEntity, buyer]",Región de Los Ríos,Chile,692001001,procuringEntity,buyer
3,id-0.3.parties.0,id-0.3,CL-MP-968521,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIE...,"[procuringEntity, buyer]",Región del Maule,Chile,616069128,procuringEntity,buyer
4,id-0.3.parties.1,id-0.3,CL-MP-1011396,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD,"[tenderer, nan]",Región Metropolitana de Santiago,Chile,768357935,tenderer,NaN


In [61]:
df_G = df_parties.dropna(subset=["address_region"])
df_G = df_G[df_G["address_countryName"].isna()]

In [62]:
df_G["address_region"].unique()

array(['Región Metropolitana de Santiago', 'Región del Maule ',
       'Región de Tarapacá  ', 'Región de Antofagasta ',
       'Región del Biobío ', 'Región de Los Ríos',
       'Región de la Araucanía ', 'Región de Valparaíso ',
       'Región de Atacama ', 'Región de Coquimbo ',
       'Región de los Lagos ', 'Región de Magallanes y de la Antártica',
       'Región de Arica y Parinacota',
       'Región del Libertador General Bernardo O´Higgins',
       'Región Aysén del General Carlos Ibáñez del Campo'], dtype=object)

In [63]:
df_parties["address_countryName"] = df_parties["address_countryName"].replace(np.nan, "no")
df_parties["address_region"] = df_parties["address_region"].replace(np.nan, "no")

In [64]:
def dar_pais_segun_region(indice:int, comparador="no") -> str:
    if df_parties.iloc[indice]["address_countryName"] == comparador:
        if df_parties.iloc[indice]["address_region"] in set(['Región de Valparaíso ', 'Región del Biobío ','Región Metropolitana de Santiago','Región de Magallanes y de la Antártica','Región del Libertador General Bernardo O´Higgins','Región de Atacama ', 'Región de Coquimbo ',
       'Región de la Araucanía ', 'Región de los Lagos ','Región del Maule ', 'Región de Los Ríos','Región de Antofagasta ', 'Región de Tarapacá  ','Región de Arica y Parinacota','Región Aysén del General Carlos Ibáñez del Campo']):
            return ("Chile")
        else:
            return ("no")
    else:
        return df_parties.iloc[indice]["address_countryName"]

In [65]:
def dar_region_segun_pais(indice:int, comparador="no") -> str:
    if df_parties.iloc[indice]["address_region"] == comparador:
        if df_parties.iloc[indice]["address_countryName"] != "Chile" and df_parties.iloc[indice]["address_countryName"] != "no":
            return("Extranjero")
        else:
            return np.nan
    else:
        return df_parties.iloc[indice]["address_region"]

In [66]:
df_parties["address_countryName"] = df_parties.index.map(dar_pais_segun_region)

In [67]:
df_parties["address_region"] = df_parties.index.map(dar_region_segun_pais)

In [68]:
df_parties["address_region"].unique()

array(['Región Metropolitana de Santiago', 'Región del Ñuble',
       'Región de Los Ríos', 'Región del Maule ', 'Región de Valparaíso ',
       'Región del Biobío ',
       'Región del Libertador General Bernardo O´Higgins',
       'Región de los Lagos ', 'Región de la Araucanía ',
       'Región de Tarapacá  ', 'Región de Atacama ',
       'Región de Antofagasta ', nan,
       'Región Aysén del General Carlos Ibáñez del Campo',
       'Región de Coquimbo ', 'Región de Arica y Parinacota',
       'Región de Magallanes y de la Antártica', 'Extranjero'],
      dtype=object)

In [69]:
df_parties["address_countryName"].replace("no", np.nan, inplace=True)
df_parties["address_countryName"].unique()

C:\Users\camil\AppData\Local\Temp\ipykernel_18536\2455676881.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_parties["address_countryName"].replace("no", np.nan, inplace=True)


array(['Chile', nan, 'Uruguay', 'España', 'Argentina', 'Estados Unidos',
       'Francia', 'China', 'República Checa', 'Italia', 'Brasil',
       'Francia, Metropolitana', 'Perú', 'Bolivia', 'Nueva Zelanda',
       'Haití', 'Alemania', 'Colombia', 'Ecuador', 'Venezuela', 'Suecia',
       'Austria', 'Lituania', 'Antigua y Barbuda', 'Estonia',
       'Emiratos Árabes Unidos', 'México'], dtype=object)

In [70]:
df_parties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606887 entries, 0 to 606886
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   _link                606887 non-null  object
 1   _link_main           606887 non-null  object
 2   id                   606887 non-null  object
 3   name                 606887 non-null  object
 4   roles                606887 non-null  object
 5   address_region       598332 non-null  object
 6   address_countryName  598333 non-null  object
 7   identifier_id        606430 non-null  object
 8   rol 1                606887 non-null  object
 9   rol 2                269155 non-null  object
dtypes: object(10)
memory usage: 46.3+ MB


In [71]:
def dejar_rut(texto:str) -> str:
    if "-" in texto:
        return int(texto.split("-")[2])
    else:
        return np.nan

In [72]:
df_parties["id"] = df_parties["id"].apply(dejar_rut)

In [73]:
df_parties.drop(columns=["roles"], inplace=True)

## Revisión de parties_additionalIdentifiers.csv

In [74]:
camino = path.join("Chile","2021", "parties_additionalIdentifiers.csv")
df_parties_add = pd.read_csv(camino, sep=",")
df_parties_add.head()

,_link,_link_parties,_link_main,id,uri,scheme,legalName
0,id-0.0.parties.0.additionalIdentifiers.0,id-0.0.parties.0,id-0.0,1057501,https://apis.mercadopublico.cl/OCDS/data/compr...,CL-MP,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO
1,id-0.1.parties.0.additionalIdentifiers.0,id-0.1.parties.0,id-0.1,1039043,https://apis.mercadopublico.cl/OCDS/data/compr...,CL-MP,INSTITUTO DE DESARROLLO AGROPECUARIO
2,id-0.2.parties.0.additionalIdentifiers.0,id-0.2.parties.0,id-0.2,3275,https://apis.mercadopublico.cl/OCDS/data/compr...,CL-MP,I MUNICIPALIDAD VALDIVIA
3,id-0.3.parties.0.additionalIdentifiers.0,id-0.3.parties.0,id-0.3,968521,https://apis.mercadopublico.cl/OCDS/data/compr...,CL-MP,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIER
4,id-0.3.parties.1.additionalIdentifiers.0,id-0.3.parties.1,id-0.3,1011396,https://apis.mercadopublico.cl/OCDS/data/prove...,CL-MP,BEFORE PUBLICIDAD SPA


In [75]:
df_parties_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606887 entries, 0 to 606886
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   _link          606887 non-null  object
 1   _link_parties  606887 non-null  object
 2   _link_main     606887 non-null  object
 3   id             606887 non-null  int64 
 4   uri            606887 non-null  object
 5   scheme         606887 non-null  object
 6   legalName      606715 non-null  object
dtypes: int64(1), object(6)
memory usage: 32.4+ MB


In [76]:
df_parties_add.iloc[3]["uri"]

'https://apis.mercadopublico.cl/OCDS/data/comprador/unidad/968521'

Los links de uri no aportan información extra, por lo no se mantendran.

In [77]:
df_parties_add.drop(columns=["uri", "scheme", "_link_main"],inplace=True)

### Revisión de tender_items.csv

In [78]:
camino = path.join("Chile","2021", "tender_items.csv")
df_tender_item = pd.read_csv(camino, sep=",")
df_tender_item.head()

,_link,_link_main,id,description,quantity,unit_name,classification_id,classification_uri,classification_scheme
0,id-0.0.tender.items.0,id-0.0,39594375,Equipamiento y suministros médicos / Productos...,1.0,Unidad,42292904,https://apis.mercadopublico.cl/OCDS/data/produ...,UNSPSC
1,id-0.0.tender.items.1,id-0.0,39594376,Equipamiento y suministros médicos / Suministr...,1.0,Kit,42142502,https://apis.mercadopublico.cl/OCDS/data/produ...,UNSPSC
2,id-0.0.tender.items.2,id-0.0,39594377,Equipamiento y suministros médicos / Suministr...,1.0,Kit,42142502,https://apis.mercadopublico.cl/OCDS/data/produ...,UNSPSC
3,id-0.0.tender.items.3,id-0.0,39594395,Equipamiento y suministros médicos / Productos...,1.0,Unidad,42291620,https://apis.mercadopublico.cl/OCDS/data/produ...,UNSPSC
4,id-0.0.tender.items.4,id-0.0,39594396,Equipamiento y suministros médicos / Productos...,1.0,Unidad,42203403,https://apis.mercadopublico.cl/OCDS/data/produ...,UNSPSC


In [79]:
df_tender_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594076 entries, 0 to 594075
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   _link                  594076 non-null  object 
 1   _link_main             594076 non-null  object 
 2   id                     594076 non-null  int64  
 3   description            594076 non-null  object 
 4   quantity               594076 non-null  float64
 5   unit_name              594076 non-null  object 
 6   classification_id      594076 non-null  int64  
 7   classification_uri     594076 non-null  object 
 8   classification_scheme  594076 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 40.8+ MB


In [80]:
df_tender_item.drop(columns=["classification_uri"], inplace=True)

In [81]:
df_tender_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594076 entries, 0 to 594075
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   _link                  594076 non-null  object 
 1   _link_main             594076 non-null  object 
 2   id                     594076 non-null  int64  
 3   description            594076 non-null  object 
 4   quantity               594076 non-null  float64
 5   unit_name              594076 non-null  object 
 6   classification_id      594076 non-null  int64  
 7   classification_scheme  594076 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 36.3+ MB


### Revisión de tender_tenderers.csv

In [82]:
camino = path.join("Chile","2021", "tender_tenderers.csv")
df_tender_tenderers = pd.read_csv(camino, sep=",")

In [83]:
df_tender_tenderers.head()

,_link,_link_main,id,name
0,id-0.3.tender.tenderers.0,id-0.3,CL-MP-1011396,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD
1,id-0.3.tender.tenderers.1,id-0.3,CL-MP-1093345,MARPRO CHILE SPA. | JAVIERA
2,id-0.3.tender.tenderers.2,id-0.3,CL-MP-1156776,COMERCIALIZADORA DE ARTÍCULOS DE ASEO JENNIFFE...
3,id-0.3.tender.tenderers.3,id-0.3,CL-MP-275627,ETHON PHARMACEUTICALS COMERCIALIZADORA IMP E...
4,id-0.3.tender.tenderers.4,id-0.3,CL-MP-38669,Crisvert Limitada | Crisvert Limitada


In [84]:
df_tender_tenderers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472184 entries, 0 to 472183
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   _link       472184 non-null  object
 1   _link_main  472184 non-null  object
 2   id          472184 non-null  object
 3   name        472184 non-null  object
dtypes: object(4)
memory usage: 14.4+ MB


In [85]:
def dejar_rut(texto:str) -> str:
    if "-" in texto:
        return int(texto.split("-")[2])
    else:
        return np.nan

In [86]:
df_tender_tenderers["id"] = df_tender_tenderers["id"].apply(dejar_rut)

In [87]:
df_tender_tenderers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472184 entries, 0 to 472183
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   _link       472184 non-null  object
 1   _link_main  472184 non-null  object
 2   id          472184 non-null  int64 
 3   name        472184 non-null  object
dtypes: int64(1), object(3)
memory usage: 14.4+ MB


### Clasificación con UNSPCS.

In [88]:
df_clasificaciones = pd.read_excel("unspcs-clasificador-de-bienes-y-servicios-de-naciones-unidas-en-espanol.xlsx", skiprows=5)
df_clasificaciones.head(10)

,Código segmento,Nombre Segmento,Código Familia,Nombre Familia,Código Clase,Nombre Clase,Código Producto,Nombre Producto
0,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101501,Gatos
1,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101502,Perros
2,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101504,Visón
3,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101505,Ratas
4,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101506,Caballos
5,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101507,Ovejas
6,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101508,Cabras
7,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101509,Asnos
8,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101510,Ratones
9,10,"Material Vivo Vegetal y Animal, Accesorios y S...",1010,Animales vivos,101015,Animales de granja,10101511,Cerdos


El dataframe df_clasificaciones corresponde a un conjunto de datos que posee el significado de cada objeto solicitados según la codificación UNSPSC, por lo que se hara merge con el df_main para saber que se esta comprando en cada licitación, se desecharán algunas de las columnas de df_clasificaciones para esto.

In [89]:
df_clasificaciones.drop(columns=["Código segmento","Código Familia","Nombre Familia","Nombre Clase","Código Clase"], inplace=True)

### Instituciones que realizan licitaciones. 

Se obtuvo unb dataset con información de los organismos que han realizado lictaciones historicamente desde <https://datos-abiertos.chilecompra.cl/descargas/complementos>, se utilizará para determinar el sector del comprador.

In [90]:
camino = path.join("Chile","Instituciones_compradoras.csv")
df_instituciones_compradoras = pd.read_csv(camino, sep=";", encoding="UTF-8")

In [91]:
df_instituciones_compradoras.head()

,sector,Instituci�n,C�digo Instituci�n,RUT Unidad de Compra,C�digo Unidad de Compra,Unidad de Compra,Regi�n Unidad de Compra
0,MUNICIPALIDADES,I MUNICIPALIDAD DE AYSEN,100049,69.240.100-K,3701,I.MUNICIPALIDAD AYSEN-DIRECCION DE EDUCACION,Ays�n
1,MUNICIPALIDADES,I MUNICIPALIDAD DE AYSEN,100049,69.240.100-K,3702,I.MUNICIPALIDAD AYSEN-ADQUISICIONES,Ays�n
2,MUNICIPALIDADES,I MUNICIPALIDAD DE AYSEN,100049,60.920.753-1,3703,I.MUNICIPALIDAD AYSEN-LICEO POLITECNICO,Ays�n
3,MUNICIPALIDADES,I MUNICIPALIDAD DE PIRQUE,100072,69.072.200-3,3704,MUNICIPALIDAD DE PIRQUE - DAF,Metropolitana
4,MUNICIPALIDADES,I MUNICIPALIDAD DE PIRQUE,100072,69.072.200-3,3705,MUNICIPALIDAD DE PIRQUE - DOM,Metropolitana


In [92]:
df_instituciones_compradoras.loc[df_instituciones_compradoras["C�digo Unidad de Compra"] == 5772]

,sector,Instituci�n,C�digo Instituci�n,RUT Unidad de Compra,C�digo Unidad de Compra,Unidad de Compra,Regi�n Unidad de Compra
2446,MUNICIPALIDADES,I MUNICIPALIDAD DE TIRUA,183037,69.160.700-3,5772,FINANZAS,B�o-B�o


In [93]:
df_instituciones_compradoras.drop(columns=["Instituci�n","Regi�n Unidad de Compra", "RUT Unidad de Compra", "C�digo Instituci�n"], inplace=True)
df_instituciones_compradoras.rename(columns={"C�digo Unidad de Compra":"Codigo Unidad de Compra"}, inplace=True)
df_instituciones_compradoras.head()

,sector,Codigo Unidad de Compra,Unidad de Compra
0,MUNICIPALIDADES,3701,I.MUNICIPALIDAD AYSEN-DIRECCION DE EDUCACION
1,MUNICIPALIDADES,3702,I.MUNICIPALIDAD AYSEN-ADQUISICIONES
2,MUNICIPALIDADES,3703,I.MUNICIPALIDAD AYSEN-LICEO POLITECNICO
3,MUNICIPALIDADES,3704,MUNICIPALIDAD DE PIRQUE - DAF
4,MUNICIPALIDADES,3705,MUNICIPALIDAD DE PIRQUE - DOM


### Unión de dataframes.

In [94]:
df_filtrado.head()

,_link,id,initiationType,tender_id,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,tender_tenderPeriod_startDate,tender_tenderPeriod_durationInDays,tender_enquiryPeriod_endDate,tender_enquiryPeriod_startDate,tender_enquiryPeriod_durationInDays,tender_procuringEntity_name,estimated_cost,buyer_id,buyer_name,estimated_cost_monetary_unit
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,2021-12-02 19:30:51,4 days 19:29:09,2021-12-04 18:00:00,2021-12-02 19:30:51,1 days 22:29:09,DIRECCION DE LOGISTICA DE CARABINEROS | SECCIÓ...,5000000.0,1079639,NaN,CLP
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,2021-12-02 18:01:35,6 days 20:59:25,2021-12-06 10:00:00,2021-12-02 18:06:00,3 days 15:54:00,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,NaN,1058095,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,NaN
2,id-0.10,ocds-70d2nz-4993-132-L121-2021-12-22T17:49:37Z,tender,4993-132-L121,L1,unsuccessful,open,False,2021-12-16 15:41:00,2021-12-15 15:41:00,...,2021-12-09 13:55:02,6 days 01:44:58,2021-12-12 17:40:00,2021-12-09 17:40:00,3 days 00:00:00,MUNICIPALIDAD DE CHOLCHOL | Secplan,500200.0,5906,NaN,CLP
3,id-0.11,ocds-70d2nz-4019-53-L121-2021-12-14T19:52:56Z,tender,4019-53-L121,L1,complete,open,True,2021-12-13 15:01:00,2021-12-10 15:01:00,...,2021-12-03 22:18:00,6 days 16:43:00,2021-12-06 23:59:00,2021-12-03 23:59:00,3 days 00:00:00,ILUSTRE MUNICIPALIDAD DE CAMINA | Depto. de Fi...,NaN,4939,NaN,NaN
4,id-0.14,ocds-70d2nz-780724-18-LE21-2021-12-22T18:09:39Z,tender,780724-18-LE21,LE,complete,open,True,2021-12-23 18:00:00,2021-12-20 15:15:00,...,2021-12-10 15:52:15,9 days 23:07:45,2021-12-15 23:59:00,2021-12-10 18:01:00,5 days 05:58:00,ILUSTRE TERCER TRIBUNAL AMBIENTAL | ILUSTRE TE...,33000000.0,780724,NaN,CLP


In [95]:
df_awards.head()

,_link,_link_main,date,status,value_amount,value_monetary_unit
0,id-0.0.awards.0,id-0.0,2017-12-23T13:23:24Z,active,690000.0,CLP
1,id-0.1.awards.0,id-0.1,NaN,unsuccessful,NaN,NaN
2,id-0.2.awards.0,id-0.2,2018-02-02T11:16:54Z,active,870000.0,CLP
3,id-0.3.awards.0,id-0.3,2017-12-26T18:19:33Z,unsuccessful,NaN,NaN
4,id-0.4.awards.0,id-0.4,2017-12-21T18:17:23Z,active,6000000.0,CLP


In [96]:
df_awards_items.head()

,_link,_link_awards,_link_main,id,quantity,unit_name,unit_value_amount,classification_id,unit_value_monetary_unit
0,id-0.9.awards.0.items.0,id-0.9.awards.0,id-0.9,39695225,1540.0,Ampolla,5500.0,51211612,CLP
1,id-0.9.awards.0.items.1,id-0.9.awards.0,id-0.9,39695228,2500.0,Ampolla,840.0,51151727,CLP
2,id-0.9.awards.0.items.2,id-0.9.awards.0,id-0.9,39695237,300.0,Ampolla,17239.0,51151823,CLP
3,id-0.9.awards.0.items.3,id-0.9.awards.0,id-0.9,39695240,28000.0,Unidad,300.0,51171504,CLP
4,id-0.9.awards.0.items.4,id-0.9.awards.0,id-0.9,39695243,200.0,Frasco,8900.0,51142206,CLP


In [97]:
df_awards_supplier.head()

,_link,_link_awards,_link_main,id,name
0,id-0.9.awards.0.suppliers.0,id-0.9.awards.0,id-0.9,106395,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.
1,id-0.9.awards.0.suppliers.1,id-0.9.awards.0,id-0.9,24938,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA
2,id-0.9.awards.0.suppliers.2,id-0.9.awards.0,id-0.9,44934,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...
3,id-0.9.awards.0.suppliers.3,id-0.9.awards.0,id-0.9,54079,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.
4,id-0.9.awards.0.suppliers.4,id-0.9.awards.0,id-0.9,747806,Indopharma S.A. | Indopharma S.A.


In [98]:
df_parties.head()

,_link,_link_main,id,name,address_region,address_countryName,identifier_id,rol 1,rol 2
0,id-0.0.parties.0,id-0.0,1057501,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Bien...,Región Metropolitana de Santiago,Chile,616085026,procuringEntity,buyer
1,id-0.1.parties.0,id-0.1,1039043,INSTITUTO DE DESARROLLO AGROPECUARIO | INSTITU...,Región del Ñuble,Chile,613070001,procuringEntity,buyer
2,id-0.2.parties.0,id-0.2,3275,I MUNICIPALIDAD VALDIVIA | UNIDAD ADQUISICIONE...,Región de Los Ríos,Chile,692001001,procuringEntity,buyer
3,id-0.3.parties.0,id-0.3,968521,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIE...,Región del Maule,Chile,616069128,procuringEntity,buyer
4,id-0.3.parties.1,id-0.3,1011396,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD,Región Metropolitana de Santiago,Chile,768357935,tenderer,NaN


In [99]:
df_parties_add.head()

,_link,_link_parties,id,legalName
0,id-0.0.parties.0.additionalIdentifiers.0,id-0.0.parties.0,1057501,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO
1,id-0.1.parties.0.additionalIdentifiers.0,id-0.1.parties.0,1039043,INSTITUTO DE DESARROLLO AGROPECUARIO
2,id-0.2.parties.0.additionalIdentifiers.0,id-0.2.parties.0,3275,I MUNICIPALIDAD VALDIVIA
3,id-0.3.parties.0.additionalIdentifiers.0,id-0.3.parties.0,968521,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIER
4,id-0.3.parties.1.additionalIdentifiers.0,id-0.3.parties.1,1011396,BEFORE PUBLICIDAD SPA


In [100]:
df_tender_item.head()

,_link,_link_main,id,description,quantity,unit_name,classification_id,classification_scheme
0,id-0.0.tender.items.0,id-0.0,39594375,Equipamiento y suministros médicos / Productos...,1.0,Unidad,42292904,UNSPSC
1,id-0.0.tender.items.1,id-0.0,39594376,Equipamiento y suministros médicos / Suministr...,1.0,Kit,42142502,UNSPSC
2,id-0.0.tender.items.2,id-0.0,39594377,Equipamiento y suministros médicos / Suministr...,1.0,Kit,42142502,UNSPSC
3,id-0.0.tender.items.3,id-0.0,39594395,Equipamiento y suministros médicos / Productos...,1.0,Unidad,42291620,UNSPSC
4,id-0.0.tender.items.4,id-0.0,39594396,Equipamiento y suministros médicos / Productos...,1.0,Unidad,42203403,UNSPSC


In [101]:
df_tender_tenderers.head()

,_link,_link_main,id,name
0,id-0.3.tender.tenderers.0,id-0.3,1011396,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD
1,id-0.3.tender.tenderers.1,id-0.3,1093345,MARPRO CHILE SPA. | JAVIERA
2,id-0.3.tender.tenderers.2,id-0.3,1156776,COMERCIALIZADORA DE ARTÍCULOS DE ASEO JENNIFFE...
3,id-0.3.tender.tenderers.3,id-0.3,275627,ETHON PHARMACEUTICALS COMERCIALIZADORA IMP E...
4,id-0.3.tender.tenderers.4,id-0.3,38669,Crisvert Limitada | Crisvert Limitada


In [102]:
df_clasificaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49022 entries, 0 to 49021
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Nombre Segmento  49022 non-null  object
 1   Código Producto  49022 non-null  int64 
 2   Nombre Producto  49022 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


### Award_items y clasificaciones.

In [103]:
df_award_item_clasifi = df_awards_items.merge(df_clasificaciones, left_on="classification_id",right_on="Código Producto")
df_award_item_clasifi.drop(columns=[ "_link"], inplace=True)
df_award_item_clasifi.head()

,_link_awards,_link_main,id,quantity,unit_name,unit_value_amount,classification_id,unit_value_monetary_unit,Nombre Segmento,Código Producto,Nombre Producto
0,id-0.9.awards.0,id-0.9,39695225,1540.0,Ampolla,5500.0,51211612,CLP,Medicamentos y Productos Farmacéuticos,51211612,Leucovorina
1,id-0.9.awards.0,id-0.9,39695228,2500.0,Ampolla,840.0,51151727,CLP,Medicamentos y Productos Farmacéuticos,51151727,Norepinefrina bitartrato
2,id-0.9.awards.0,id-0.9,39695237,300.0,Ampolla,17239.0,51151823,CLP,Medicamentos y Productos Farmacéuticos,51151823,Hidrocloruro de labetalol
3,id-0.9.awards.0,id-0.9,39695240,28000.0,Unidad,300.0,51171504,CLP,Medicamentos y Productos Farmacéuticos,51171504,Antiácidos de bicarbonato de sodio
4,id-0.9.awards.0,id-0.9,39695243,200.0,Frasco,8900.0,51142206,CLP,Medicamentos y Productos Farmacéuticos,51142206,Sulfato de morfina


### Awards y Awards_items

In [104]:
df_award_and_items = df_awards.merge(df_award_item_clasifi, left_on="_link", right_on="_link_awards")
df_award_and_items.head()

,_link,_link_main_x,date,status,value_amount,value_monetary_unit,_link_awards,_link_main_y,id,quantity,unit_name,unit_value_amount,classification_id,unit_value_monetary_unit,Nombre Segmento,Código Producto,Nombre Producto
0,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,id-0.9.awards.0,id-0.9,39695225,1540.0,Ampolla,5500.0,51211612,CLP,Medicamentos y Productos Farmacéuticos,51211612,Leucovorina
1,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,id-0.9.awards.0,id-0.9,39695228,2500.0,Ampolla,840.0,51151727,CLP,Medicamentos y Productos Farmacéuticos,51151727,Norepinefrina bitartrato
2,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,id-0.9.awards.0,id-0.9,39695237,300.0,Ampolla,17239.0,51151823,CLP,Medicamentos y Productos Farmacéuticos,51151823,Hidrocloruro de labetalol
3,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,id-0.9.awards.0,id-0.9,39695240,28000.0,Unidad,300.0,51171504,CLP,Medicamentos y Productos Farmacéuticos,51171504,Antiácidos de bicarbonato de sodio
4,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,id-0.9.awards.0,id-0.9,39695243,200.0,Frasco,8900.0,51142206,CLP,Medicamentos y Productos Farmacéuticos,51142206,Sulfato de morfina


In [105]:
df_award_and_items.drop(columns=["_link_main_y", "_link_awards"],inplace=True)

In [106]:
df_igualdad = pd.DataFrame(df_award_and_items["value_monetary_unit"] == df_award_and_items["unit_value_monetary_unit"])
indice = df_igualdad.loc[df_igualdad[0] == False].index

In [107]:
df_award_and_items.iloc[indice]

,_link,_link_main_x,date,status,value_amount,value_monetary_unit,id,quantity,unit_name,unit_value_amount,classification_id,unit_value_monetary_unit,Nombre Segmento,Código Producto,Nombre Producto
0,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,39695225,1540.0,Ampolla,5500.0,51211612,CLP,Medicamentos y Productos Farmacéuticos,51211612,Leucovorina
1,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,39695228,2500.0,Ampolla,840.0,51151727,CLP,Medicamentos y Productos Farmacéuticos,51151727,Norepinefrina bitartrato
2,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,39695237,300.0,Ampolla,17239.0,51151823,CLP,Medicamentos y Productos Farmacéuticos,51151823,Hidrocloruro de labetalol
3,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,39695240,28000.0,Unidad,300.0,51171504,CLP,Medicamentos y Productos Farmacéuticos,51171504,Antiácidos de bicarbonato de sodio
4,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,39695243,200.0,Frasco,8900.0,51142206,CLP,Medicamentos y Productos Farmacéuticos,51142206,Sulfato de morfina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312277,id-15.8747.awards.0,id-15.8747,2016-08-03T15:57:26Z,unsuccessful,NaN,NaN,38887242,1.0,Unidad,2087735.0,23153004,CLP,Maquinaria y Accesorios para Manufactura y Pro...,23153004,Plantilla de aguja
312278,id-15.8751.awards.0,id-15.8751,NaN,unsuccessful,NaN,NaN,38869355,1.0,Unidad,4154210.0,43232304,CLP,Difusión de Tecnologías de Información y Telec...,43232304,Software de sistemas de manejo de base datos
312279,id-15.8752.awards.0,id-15.8752,2016-07-05T10:06:11Z,unsuccessful,NaN,NaN,38817004,1.0,Unidad,1446107.0,76111501,CLP,"Servicios de Limpieza, Descontaminación y Trat...",76111501,Servicios de limpieza de edificios
312280,id-15.8753.awards.0,id-15.8753,NaN,unsuccessful,NaN,NaN,38819049,6.0,Mes/Hombre,6435000.0,92101902,CLP,"Servicios de Defensa Nacional, Orden Publico, ...",92101902,Servicios de ambulancia


In [108]:
df_award_and_items.drop(indice, inplace=True)

In [109]:
df_award_and_items.rename({"_link_main_x":"_link_main"},inplace=True)

In [110]:
df_award_and_items.columns

Index(['_link', '_link_main_x', 'date', 'status', 'value_amount',
       'value_monetary_unit', 'id', 'quantity', 'unit_name',
       'unit_value_amount', 'classification_id', 'unit_value_monetary_unit',
       'Nombre Segmento', 'Código Producto', 'Nombre Producto'],
      dtype='object')

### Awards y Awards_supplier

In [111]:
df_award_and_supplier = df_awards.merge(df_awards_supplier, left_on="_link", right_on="_link_awards")
df_award_and_supplier.drop(columns=["_link_awards", "_link_main_y", "_link_y"], inplace=True)
df_award_and_supplier.head()

,_link_x,_link_main_x,date,status,value_amount,value_monetary_unit,id,name
0,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,106395,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.
1,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,24938,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA
2,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,44934,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...
3,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,54079,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.
4,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,747806,Indopharma S.A. | Indopharma S.A.


In [112]:
df_award_and_supplier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131445 entries, 0 to 131444
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _link_x              131445 non-null  object 
 1   _link_main_x         131445 non-null  object 
 2   date                 114496 non-null  object 
 3   status               131445 non-null  object 
 4   value_amount         96148 non-null   float64
 5   value_monetary_unit  96148 non-null   object 
 6   id                   131445 non-null  int64  
 7   name                 131445 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 8.0+ MB


#### Tender_items y Clasificaciones.

In [113]:
df_tender_item = df_tender_item.merge(df_clasificaciones, left_on="classification_id",right_on="Código Producto")
df_tender_item.drop(columns=["classification_scheme", "classification_id"], inplace=True)
df_tender_item.head()

,_link,_link_main,id,description,quantity,unit_name,Nombre Segmento,Código Producto,Nombre Producto
0,id-0.0.tender.items.0,id-0.0,39594375,Equipamiento y suministros médicos / Productos...,1.0,Unidad,"Equipo Médico, Accesorios y Suministros",42292904,Suturas quirúrgicas o pasadores de alambre o p...
1,id-0.0.tender.items.1,id-0.0,39594376,Equipamiento y suministros médicos / Suministr...,1.0,Kit,"Equipo Médico, Accesorios y Suministros",42142502,Agujas para anestesia
2,id-0.0.tender.items.2,id-0.0,39594377,Equipamiento y suministros médicos / Suministr...,1.0,Kit,"Equipo Médico, Accesorios y Suministros",42142502,Agujas para anestesia
3,id-0.0.tender.items.3,id-0.0,39594395,Equipamiento y suministros médicos / Productos...,1.0,Unidad,"Equipo Médico, Accesorios y Suministros",42291620,Trocadores quirúrgicos para uso general o acce...
4,id-0.0.tender.items.4,id-0.0,39594396,Equipamiento y suministros médicos / Productos...,1.0,Unidad,"Equipo Médico, Accesorios y Suministros",42203403,Introductores de catéteres o sets de diagnósti...


### Awards y Tender_items_clasificaciones.

In [114]:
df_items_tender_awards = df_award_and_items.merge(df_tender_item, left_on=["_link_main_x","classification_id","unit_name"], right_on=["_link_main","Código Producto","unit_name"], how="left")

In [115]:
df_items_tender_awards.head()

,_link_x,_link_main_x,date,status,value_amount,value_monetary_unit,id_x,quantity_x,unit_name,unit_value_amount,...,Código Producto_x,Nombre Producto_x,_link_y,_link_main,id_y,description,quantity_y,Nombre Segmento_y,Código Producto_y,Nombre Producto_y
0,id-0.11.awards.0,id-0.11,2018-01-08T11:20:54Z,active,12605000.0,CLP,39705329,1.0,Unidad,3500000.0,...,70122008,Tecnología para laboratorio veterinario,id-0.11.tender.items.0,id-0.11,39705329,"Servicios agrícolas, pesqueros, forestales y r...",1.0,"Servicios de Contratación Agrícola, Pesquera, ...",70122008,Tecnología para laboratorio veterinario
1,id-0.14.awards.0,id-0.14,2017-12-22T14:17:12Z,active,890400.0,CLP,39633115,12.0,Unidad,2294900.0,...,76111501,Servicios de limpieza de edificios,id-0.14.tender.items.0,id-0.14,39633115,Servicios de limpieza industrial / Servicios d...,1.0,"Servicios de Limpieza, Descontaminación y Trat...",76111501,Servicios de limpieza de edificios
2,id-0.15.awards.0,id-0.15,2018-01-04T15:35:26Z,active,1512605.0,CLP,39703470,200.0,Unidad,895.0,...,53121603,Morrales,id-0.15.tender.items.0,id-0.15,39703470,"Ropa, maletas y productos de aseo personal / M...",200.0,"Ropa, Maletas y Productos de Aseo Personal",53121603,Morrales
3,id-0.15.awards.0,id-0.15,2018-01-04T15:35:26Z,active,1512605.0,CLP,39703482,120.0,Unidad,5871.0,...,52121702,Toallas playeras,id-0.15.tender.items.1,id-0.15,39703482,"Muebles, accesorios, electrodomésticos y produ...",120.0,"Artículos Domésticos, Suministros y Productos ...",52121702,Toallas playeras
4,id-0.15.awards.0,id-0.15,2018-01-04T15:35:26Z,active,1512605.0,CLP,39703553,125.0,Unidad,1984.0,...,53102503,Sombreros,id-0.15.tender.items.2,id-0.15,39703553,"Ropa, maletas y productos de aseo personal / R...",125.0,"Ropa, Maletas y Productos de Aseo Personal",53102503,Sombreros


In [116]:
df_items_tender_awards.columns

Index(['_link_x', '_link_main_x', 'date', 'status', 'value_amount',
       'value_monetary_unit', 'id_x', 'quantity_x', 'unit_name',
       'unit_value_amount', 'classification_id', 'unit_value_monetary_unit',
       'Nombre Segmento_x', 'Código Producto_x', 'Nombre Producto_x',
       '_link_y', '_link_main', 'id_y', 'description', 'quantity_y',
       'Nombre Segmento_y', 'Código Producto_y', 'Nombre Producto_y'],
      dtype='object')

In [117]:
df_items_tender_awards.drop(columns=['_link_y', '_link_main','quantity_x', 'classification_id', "id_x"], inplace=True)
df_items_tender_awards.rename(columns={'_link_x':"_link_awards", '_link_main_x':"_link_main", 'date':"date_award", 'status':"award_status",'quantity_x':"quantity", 'unit_name_x':"unit_name", "id_y":"award_id"}, inplace=True)
df_items_tender_awards.head()

,_link_awards,_link_main,date_award,award_status,value_amount,value_monetary_unit,unit_name,unit_value_amount,unit_value_monetary_unit,Nombre Segmento_x,Código Producto_x,Nombre Producto_x,award_id,description,quantity_y,Nombre Segmento_y,Código Producto_y,Nombre Producto_y
0,id-0.11.awards.0,id-0.11,2018-01-08T11:20:54Z,active,12605000.0,CLP,Unidad,3500000.0,CLP,"Servicios de Contratación Agrícola, Pesquera, ...",70122008,Tecnología para laboratorio veterinario,39705329,"Servicios agrícolas, pesqueros, forestales y r...",1.0,"Servicios de Contratación Agrícola, Pesquera, ...",70122008,Tecnología para laboratorio veterinario
1,id-0.14.awards.0,id-0.14,2017-12-22T14:17:12Z,active,890400.0,CLP,Unidad,2294900.0,CLP,"Servicios de Limpieza, Descontaminación y Trat...",76111501,Servicios de limpieza de edificios,39633115,Servicios de limpieza industrial / Servicios d...,1.0,"Servicios de Limpieza, Descontaminación y Trat...",76111501,Servicios de limpieza de edificios
2,id-0.15.awards.0,id-0.15,2018-01-04T15:35:26Z,active,1512605.0,CLP,Unidad,895.0,CLP,"Ropa, Maletas y Productos de Aseo Personal",53121603,Morrales,39703470,"Ropa, maletas y productos de aseo personal / M...",200.0,"Ropa, Maletas y Productos de Aseo Personal",53121603,Morrales
3,id-0.15.awards.0,id-0.15,2018-01-04T15:35:26Z,active,1512605.0,CLP,Unidad,5871.0,CLP,"Artículos Domésticos, Suministros y Productos ...",52121702,Toallas playeras,39703482,"Muebles, accesorios, electrodomésticos y produ...",120.0,"Artículos Domésticos, Suministros y Productos ...",52121702,Toallas playeras
4,id-0.15.awards.0,id-0.15,2018-01-04T15:35:26Z,active,1512605.0,CLP,Unidad,1984.0,CLP,"Ropa, Maletas y Productos de Aseo Personal",53102503,Sombreros,39703553,"Ropa, maletas y productos de aseo personal / R...",125.0,"Ropa, Maletas y Productos de Aseo Personal",53102503,Sombreros


### Parties y Parties_additional_identifiers.

In [118]:
df_parties_and_add = df_parties.merge(df_parties_add, left_on=["_link","id"], right_on=["_link_parties", "id"], how="left")
df_parties_and_add.head()

,_link_x,_link_main,id,name,address_region,address_countryName,identifier_id,rol 1,rol 2,_link_y,_link_parties,legalName
0,id-0.0.parties.0,id-0.0,1057501,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Bien...,Región Metropolitana de Santiago,Chile,616085026,procuringEntity,buyer,id-0.0.parties.0.additionalIdentifiers.0,id-0.0.parties.0,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO
1,id-0.1.parties.0,id-0.1,1039043,INSTITUTO DE DESARROLLO AGROPECUARIO | INSTITU...,Región del Ñuble,Chile,613070001,procuringEntity,buyer,id-0.1.parties.0.additionalIdentifiers.0,id-0.1.parties.0,INSTITUTO DE DESARROLLO AGROPECUARIO
2,id-0.2.parties.0,id-0.2,3275,I MUNICIPALIDAD VALDIVIA | UNIDAD ADQUISICIONE...,Región de Los Ríos,Chile,692001001,procuringEntity,buyer,id-0.2.parties.0.additionalIdentifiers.0,id-0.2.parties.0,I MUNICIPALIDAD VALDIVIA
3,id-0.3.parties.0,id-0.3,968521,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIE...,Región del Maule,Chile,616069128,procuringEntity,buyer,id-0.3.parties.0.additionalIdentifiers.0,id-0.3.parties.0,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIER
4,id-0.3.parties.1,id-0.3,1011396,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD,Región Metropolitana de Santiago,Chile,768357935,tenderer,NaN,id-0.3.parties.1.additionalIdentifiers.0,id-0.3.parties.1,BEFORE PUBLICIDAD SPA


In [119]:
df_parties_and_add.drop(columns=["_link_y", "_link_parties"], inplace=True)
df_parties_and_add.head()

,_link_x,_link_main,id,name,address_region,address_countryName,identifier_id,rol 1,rol 2,legalName
0,id-0.0.parties.0,id-0.0,1057501,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Bien...,Región Metropolitana de Santiago,Chile,616085026,procuringEntity,buyer,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO
1,id-0.1.parties.0,id-0.1,1039043,INSTITUTO DE DESARROLLO AGROPECUARIO | INSTITU...,Región del Ñuble,Chile,613070001,procuringEntity,buyer,INSTITUTO DE DESARROLLO AGROPECUARIO
2,id-0.2.parties.0,id-0.2,3275,I MUNICIPALIDAD VALDIVIA | UNIDAD ADQUISICIONE...,Región de Los Ríos,Chile,692001001,procuringEntity,buyer,I MUNICIPALIDAD VALDIVIA
3,id-0.3.parties.0,id-0.3,968521,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIE...,Región del Maule,Chile,616069128,procuringEntity,buyer,SERVICIO DE SALUD DEL MAULE HOSPITAL SAN JAVIER
4,id-0.3.parties.1,id-0.3,1011396,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD,Región Metropolitana de Santiago,Chile,768357935,tenderer,NaN,BEFORE PUBLICIDAD SPA


In [120]:
df_parties_and_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606887 entries, 0 to 606886
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   _link_x              606887 non-null  object
 1   _link_main           606887 non-null  object
 2   id                   606887 non-null  int64 
 3   name                 606887 non-null  object
 4   address_region       598332 non-null  object
 5   address_countryName  598333 non-null  object
 6   identifier_id        606430 non-null  object
 7   rol 1                606887 non-null  object
 8   rol 2                269155 non-null  object
 9   legalName            606715 non-null  object
dtypes: int64(1), object(9)
memory usage: 46.3+ MB


### Union de awards_supplier y parties.

In [121]:
df_supplier_additional_info = df_award_and_supplier.merge(df_parties_and_add, left_on=["_link_main_x", "name"], right_on=["_link_main", "name"])
df_supplier_additional_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129833 entries, 0 to 129832
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _link_x_x            129833 non-null  object 
 1   _link_main_x         129833 non-null  object 
 2   date                 113046 non-null  object 
 3   status               129833 non-null  object 
 4   value_amount         94918 non-null   float64
 5   value_monetary_unit  94918 non-null   object 
 6   id_x                 129833 non-null  int64  
 7   name                 129833 non-null  object 
 8   _link_x_y            129833 non-null  object 
 9   _link_main           129833 non-null  object 
 10  id_y                 129833 non-null  int64  
 11  address_region       127471 non-null  object 
 12  address_countryName  127471 non-null  object 
 13  identifier_id        129665 non-null  object 
 14  rol 1                129833 non-null  object 
 15  rol 2            

In [122]:
df_supplier_additional_info.head()

,_link_x_x,_link_main_x,date,status,value_amount,value_monetary_unit,id_x,name,_link_x_y,_link_main,id_y,address_region,address_countryName,identifier_id,rol 1,rol 2,legalName
0,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,106395,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,id-0.9.parties.12,id-0.9,106395,Región Metropolitana de Santiago,Chile,775969407,supplier,tenderer,LABORATORIO CHILE S A
1,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,24938,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,id-0.9.parties.13,id-0.9,24938,Región del Biobío,Chile,876744007,supplier,tenderer,LABORATORIO PASTEUR S.A.
2,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,44934,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...,id-0.9.parties.7,id-0.9,44934,Región Metropolitana de Santiago,Chile,77478120K,supplier,tenderer,FRESENIUS KABI CHILE LIMITADA
3,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,54079,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.,id-0.9.parties.6,id-0.9,54079,Región del Biobío,Chile,777689908,supplier,tenderer,FARMACEUTICA INSUVAL S.A.
4,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,747806,Indopharma S.A. | Indopharma S.A.,id-0.9.parties.9,id-0.9,747806,Región de Valparaíso,Chile,762740273,supplier,tenderer,INDOPHARMA S.A.


In [123]:
df_supplier_additional_info.drop(columns=["id_y", "_link_main", "_link_x_y"], inplace=True)
df_supplier_additional_info.rename(columns={"_link_main_x":"_link_main", "_link_x_x":"_link_awards", "id_x":"supplier_id"}, inplace=True)
df_supplier_additional_info.head()

,_link_awards,_link_main,date,status,value_amount,value_monetary_unit,supplier_id,name,address_region,address_countryName,identifier_id,rol 1,rol 2,legalName
0,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,106395,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,Región Metropolitana de Santiago,Chile,775969407,supplier,tenderer,LABORATORIO CHILE S A
1,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,24938,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,Región del Biobío,Chile,876744007,supplier,tenderer,LABORATORIO PASTEUR S.A.
2,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,44934,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...,Región Metropolitana de Santiago,Chile,77478120K,supplier,tenderer,FRESENIUS KABI CHILE LIMITADA
3,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,54079,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.,Región del Biobío,Chile,777689908,supplier,tenderer,FARMACEUTICA INSUVAL S.A.
4,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,747806,Indopharma S.A. | Indopharma S.A.,Región de Valparaíso,Chile,762740273,supplier,tenderer,INDOPHARMA S.A.


## Unión de main con otros dataframes.

In [124]:
df_main_buyer_info = df_filtrado.merge(df_parties_and_add, right_on=["id","_link_main"], left_on=["buyer_id","_link"], how="left")
df_main_buyer_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116359 entries, 0 to 116358
Data columns (total 32 columns):
 #   Column                               Non-Null Count   Dtype          
---  ------                               --------------   -----          
 0   _link                                116359 non-null  object         
 1   id_x                                 116359 non-null  object         
 2   initiationType                       116359 non-null  object         
 3   tender_id                            116359 non-null  object         
 4   tender_procurementMethodDetails      115246 non-null  category       
 5   tender_status                        116359 non-null  object         
 6   tender_procurementMethod             116359 non-null  object         
 7   tender_hasEnquiries                  116359 non-null  bool           
 8   tender_awardPeriod_endDate           116359 non-null  datetime64[ns] 
 9   tender_awardPeriod_startDate         116359 non-null  datet

In [125]:
df_main_buyer_info.head()

,_link,id_x,initiationType,tender_id,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,_link_x,_link_main,id_y,name,address_region,address_countryName,identifier_id,rol 1,rol 2,legalName
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,id-0.4.parties.0,id-0.4,1079639,DIRECCION DE LOGISTICA DE CARABINEROS | SECCIÓ...,Región de los Lagos,Chile,619793005,procuringEntity,buyer,SECCIÓN COMPRAS X ZONA
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,id-0.9.parties.0,id-0.9,1058095,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,Región Metropolitana de Santiago,Chile,616085026,procuringEntity,buyer,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO
2,id-0.10,ocds-70d2nz-4993-132-L121-2021-12-22T17:49:37Z,tender,4993-132-L121,L1,unsuccessful,open,False,2021-12-16 15:41:00,2021-12-15 15:41:00,...,id-0.10.parties.0,id-0.10,5906,MUNICIPALIDAD DE CHOLCHOL | Secplan,Región de la Araucanía,Chile,69265000K,procuringEntity,buyer,MUNICIPALIDAD DE CHOLCHOL
3,id-0.11,ocds-70d2nz-4019-53-L121-2021-12-14T19:52:56Z,tender,4019-53-L121,L1,complete,open,True,2021-12-13 15:01:00,2021-12-10 15:01:00,...,id-0.11.parties.0,id-0.11,4939,ILUSTRE MUNICIPALIDAD DE CAMINA | Depto. de Fi...,Región de Tarapacá,Chile,69251100K,procuringEntity,buyer,ILUSTRE MUNICIPALIDAD DE CAMINA
4,id-0.14,ocds-70d2nz-780724-18-LE21-2021-12-22T18:09:39Z,tender,780724-18-LE21,LE,complete,open,True,2021-12-23 18:00:00,2021-12-20 15:15:00,...,id-0.14.parties.0,id-0.14,780724,ILUSTRE TERCER TRIBUNAL AMBIENTAL | ILUSTRE TE...,Región de Los Ríos,Chile,619803302,procuringEntity,buyer,ILUSTRE TERCER TRIBUNAL AMBIENTAL


In [126]:
df_main_buyer_info.rename(columns={"id_x":"tender_id", "id_y":"id"}, inplace=True)

In [127]:
df_main_buyer_info = df_main_buyer_info.merge(df_instituciones_compradoras, left_on="id", right_on="Codigo Unidad de Compra")

In [128]:
df_main_buyer_info.head()

,_link,tender_id,initiationType,tender_id,tender_procurementMethodDetails,tender_status,tender_procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,name,address_region,address_countryName,identifier_id,rol 1,rol 2,legalName,sector,Codigo Unidad de Compra,Unidad de Compra
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,DIRECCION DE LOGISTICA DE CARABINEROS | SECCIÓ...,Región de los Lagos,Chile,619793005,procuringEntity,buyer,SECCIÓN COMPRAS X ZONA,FFAA,1079639,SECCI�N COMPRAS ZONA DE CARABINEROS LOS LAGOS
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,Región Metropolitana de Santiago,Chile,616085026,procuringEntity,buyer,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO,SALUD,1058095,F�RMARCOS
2,id-0.10,ocds-70d2nz-4993-132-L121-2021-12-22T17:49:37Z,tender,4993-132-L121,L1,unsuccessful,open,False,2021-12-16 15:41:00,2021-12-15 15:41:00,...,MUNICIPALIDAD DE CHOLCHOL | Secplan,Región de la Araucanía,Chile,69265000K,procuringEntity,buyer,MUNICIPALIDAD DE CHOLCHOL,MUNICIPALIDADES,5906,SECPLAN
3,id-0.11,ocds-70d2nz-4019-53-L121-2021-12-14T19:52:56Z,tender,4019-53-L121,L1,complete,open,True,2021-12-13 15:01:00,2021-12-10 15:01:00,...,ILUSTRE MUNICIPALIDAD DE CAMINA | Depto. de Fi...,Región de Tarapacá,Chile,69251100K,procuringEntity,buyer,ILUSTRE MUNICIPALIDAD DE CAMINA,MUNICIPALIDADES,4939,DEPTO. DE FINANZAS
4,id-0.14,ocds-70d2nz-780724-18-LE21-2021-12-22T18:09:39Z,tender,780724-18-LE21,LE,complete,open,True,2021-12-23 18:00:00,2021-12-20 15:15:00,...,ILUSTRE TERCER TRIBUNAL AMBIENTAL | ILUSTRE TE...,Región de Los Ríos,Chile,619803302,procuringEntity,buyer,ILUSTRE TERCER TRIBUNAL AMBIENTAL,LEGISLATIVO Y JUDICIAL,780724,ILUSTRE TERCER TRIBUNAL AMBIENTAL


In [129]:
df_main_buyer_info.drop(columns=["buyer_id", "tender_procuringEntity_name", "_link_main","rol 1" ,"rol 2","_link_x", "address_countryName","_link_x", "_link_main"], inplace=True)
df_main_buyer_info.rename(columns={"address_region":"buyer_region", "name":"buyer_name", "id":"buyer_id", "legalName":"buyer_legalName", "tender_procurementMethodDetails":"procurement_details", "tender_procurementMethod":"procurementMethod", "address_countryName":"buyer_country"},inplace=True)
df_main_buyer_info.head()

,_link,tender_id,initiationType,tender_id,procurement_details,tender_status,procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,buyer_name,estimated_cost_monetary_unit,buyer_id,buyer_name,buyer_region,identifier_id,buyer_legalName,sector,Codigo Unidad de Compra,Unidad de Compra
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,NaN,CLP,1079639,DIRECCION DE LOGISTICA DE CARABINEROS | SECCIÓ...,Región de los Lagos,619793005,SECCIÓN COMPRAS X ZONA,FFAA,1079639,SECCI�N COMPRAS ZONA DE CARABINEROS LOS LAGOS
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,NaN,1058095,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,Región Metropolitana de Santiago,616085026,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO,SALUD,1058095,F�RMARCOS
2,id-0.10,ocds-70d2nz-4993-132-L121-2021-12-22T17:49:37Z,tender,4993-132-L121,L1,unsuccessful,open,False,2021-12-16 15:41:00,2021-12-15 15:41:00,...,NaN,CLP,5906,MUNICIPALIDAD DE CHOLCHOL | Secplan,Región de la Araucanía,69265000K,MUNICIPALIDAD DE CHOLCHOL,MUNICIPALIDADES,5906,SECPLAN
3,id-0.11,ocds-70d2nz-4019-53-L121-2021-12-14T19:52:56Z,tender,4019-53-L121,L1,complete,open,True,2021-12-13 15:01:00,2021-12-10 15:01:00,...,NaN,NaN,4939,ILUSTRE MUNICIPALIDAD DE CAMINA | Depto. de Fi...,Región de Tarapacá,69251100K,ILUSTRE MUNICIPALIDAD DE CAMINA,MUNICIPALIDADES,4939,DEPTO. DE FINANZAS
4,id-0.14,ocds-70d2nz-780724-18-LE21-2021-12-22T18:09:39Z,tender,780724-18-LE21,LE,complete,open,True,2021-12-23 18:00:00,2021-12-20 15:15:00,...,NaN,CLP,780724,ILUSTRE TERCER TRIBUNAL AMBIENTAL | ILUSTRE TE...,Región de Los Ríos,619803302,ILUSTRE TERCER TRIBUNAL AMBIENTAL,LEGISLATIVO Y JUDICIAL,780724,ILUSTRE TERCER TRIBUNAL AMBIENTAL


####  Merge de df_main_buyer con df_supplier_additional_info y df_awards

In [130]:
df_main_buyer_suplier = df_main_buyer_info.merge(df_supplier_additional_info, left_on="_link", right_on="_link_main", suffixes=("main_", "_supplier"), how="left")
df_main_buyer_suplier.head()

,_link,tender_id,initiationType,tender_id,procurement_details,tender_status,procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,value_amount,value_monetary_unit,supplier_id,name,address_region,address_countryName,identifier_id_supplier,rol 1,rol 2,legalName
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,106395.0,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,Región Metropolitana de Santiago,Chile,775969407,supplier,tenderer,LABORATORIO CHILE S A
2,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,24938.0,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,Región del Biobío,Chile,876744007,supplier,tenderer,LABORATORIO PASTEUR S.A.
3,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,44934.0,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...,Región Metropolitana de Santiago,Chile,77478120K,supplier,tenderer,FRESENIUS KABI CHILE LIMITADA
4,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,54079.0,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.,Región del Biobío,Chile,777689908,supplier,tenderer,FARMACEUTICA INSUVAL S.A.


In [131]:
df_main_buyer_suplier.columns

Index(['_link', 'tender_id', 'initiationType', 'tender_id',
       'procurement_details', 'tender_status', 'procurementMethod',
       'tender_hasEnquiries', 'tender_awardPeriod_endDate',
       'tender_awardPeriod_startDate', 'tender_awardPeriod_durationInDays',
       'tender_tenderPeriod_endDate', 'tender_tenderPeriod_startDate',
       'tender_tenderPeriod_durationInDays', 'tender_enquiryPeriod_endDate',
       'tender_enquiryPeriod_startDate', 'tender_enquiryPeriod_durationInDays',
       'estimated_cost', 'buyer_name', 'estimated_cost_monetary_unit',
       'buyer_id', 'buyer_name', 'buyer_region', 'identifier_idmain_',
       'buyer_legalName', 'sector', 'Codigo Unidad de Compra',
       'Unidad de Compra', '_link_awards', '_link_main', 'date', 'status',
       'value_amount', 'value_monetary_unit', 'supplier_id', 'name',
       'address_region', 'address_countryName', 'identifier_id_supplier',
       'rol 1', 'rol 2', 'legalName'],
      dtype='object')

In [132]:
df_main_buyer_suplier[['estimated_cost_monetary_unit', 'buyer_id', 'buyer_name','buyer_region', 'buyer_legalName', '_link_awards', '_link_main', 'date','status', 'value_amount', 'value_monetary_unit', 'supplier_id', 'name','address_region', 'address_countryName', 'rol 1', 'rol 2', 'legalName']].head()

,estimated_cost_monetary_unit,buyer_id,buyer_name,buyer_name,buyer_region,buyer_legalName,_link_awards,_link_main,date,status,value_amount,value_monetary_unit,supplier_id,name,address_region,address_countryName,rol 1,rol 2,legalName
0,CLP,1079639,NaN,DIRECCION DE LOGISTICA DE CARABINEROS | SECCIÓ...,Región de los Lagos,SECCIÓN COMPRAS X ZONA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1058095,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,Región Metropolitana de Santiago,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,106395.0,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,Región Metropolitana de Santiago,Chile,supplier,tenderer,LABORATORIO CHILE S A
2,NaN,1058095,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,Región Metropolitana de Santiago,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,24938.0,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,Región del Biobío,Chile,supplier,tenderer,LABORATORIO PASTEUR S.A.
3,NaN,1058095,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,Región Metropolitana de Santiago,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,44934.0,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...,Región Metropolitana de Santiago,Chile,supplier,tenderer,FRESENIUS KABI CHILE LIMITADA
4,NaN,1058095,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO | Fárm...,Región Metropolitana de Santiago,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,54079.0,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.,Región del Biobío,Chile,supplier,tenderer,FARMACEUTICA INSUVAL S.A.


In [133]:
df_main_buyer_suplier.rename(columns={'date':"date_award",'status':"award_status", 'value_amount':"tender_value", 'value_monetary_unit':"tender_monetary_unit", 'name':"supplier_name",'address_region':"supplier_region", 'address_countryName':"supplier_country", 'legalName':"supplier_legal_name"}, inplace=True)
df_main_buyer_suplier.head()

,_link,tender_id,initiationType,tender_id,procurement_details,tender_status,procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,tender_value,tender_monetary_unit,supplier_id,supplier_name,supplier_region,supplier_country,identifier_id_supplier,rol 1,rol 2,supplier_legal_name
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,106395.0,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,Región Metropolitana de Santiago,Chile,775969407,supplier,tenderer,LABORATORIO CHILE S A
2,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,24938.0,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,Región del Biobío,Chile,876744007,supplier,tenderer,LABORATORIO PASTEUR S.A.
3,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,44934.0,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...,Región Metropolitana de Santiago,Chile,77478120K,supplier,tenderer,FRESENIUS KABI CHILE LIMITADA
4,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,54079.0,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.,Región del Biobío,Chile,777689908,supplier,tenderer,FARMACEUTICA INSUVAL S.A.


In [134]:
df_main_buyer_suplier.drop(columns=['rol 1', 'rol 2', '_link_awards', '_link_main'], inplace=True)

In [135]:
df_main_buyer_suplier.head()

,_link,tender_id,initiationType,tender_id,procurement_details,tender_status,procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,date_award,award_status,tender_value,tender_monetary_unit,supplier_id,supplier_name,supplier_region,supplier_country,identifier_id_supplier,supplier_legal_name
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,106395.0,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,Región Metropolitana de Santiago,Chile,775969407,LABORATORIO CHILE S A
2,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,24938.0,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,Región del Biobío,Chile,876744007,LABORATORIO PASTEUR S.A.
3,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,44934.0,Fresenius Kabi Chile Ltda. | Fresenius Kabi Ch...,Región Metropolitana de Santiago,Chile,77478120K,FRESENIUS KABI CHILE LIMITADA
4,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN,54079.0,FARMACEUTICA INSUVAL S.A. | INSUVAL S.A.,Región del Biobío,Chile,777689908,FARMACEUTICA INSUVAL S.A.


In [137]:
df_main_award = df_main_buyer_info.merge(df_award_and_items, left_on="_link", right_on="_link_main_x", how="left")
df_main_award.head()

,_link_x,tender_id,initiationType,tender_id,procurement_details,tender_status,procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,value_monetary_unit,id,quantity,unit_name,unit_value_amount,classification_id,unit_value_monetary_unit,Nombre Segmento,Código Producto,Nombre Producto
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,id-0.10,ocds-70d2nz-4993-132-L121-2021-12-22T17:49:37Z,tender,4993-132-L121,L1,unsuccessful,open,False,2021-12-16 15:41:00,2021-12-15 15:41:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,id-0.11,ocds-70d2nz-4019-53-L121-2021-12-14T19:52:56Z,tender,4019-53-L121,L1,complete,open,True,2021-12-13 15:01:00,2021-12-10 15:01:00,...,CLP,39705329.0,1.0,Unidad,3500000.0,70122008.0,CLP,"Servicios de Contratación Agrícola, Pesquera, ...",70122008.0,Tecnología para laboratorio veterinario
4,id-0.14,ocds-70d2nz-780724-18-LE21-2021-12-22T18:09:39Z,tender,780724-18-LE21,LE,complete,open,True,2021-12-23 18:00:00,2021-12-20 15:15:00,...,CLP,39633115.0,12.0,Unidad,2294900.0,76111501.0,CLP,"Servicios de Limpieza, Descontaminación y Trat...",76111501.0,Servicios de limpieza de edificios


### main_award y tenderers.

In [138]:
df_tenderers = df_tender_tenderers.merge(df_parties_and_add, left_on=["_link_main","id"], right_on=["_link_main", "id"], how="left")
df_tenderers.head()

,_link,_link_main,id,name_x,_link_x,name_y,address_region,address_countryName,identifier_id,rol 1,rol 2,legalName
0,id-0.3.tender.tenderers.0,id-0.3,1011396,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD,id-0.3.parties.1,BEFORE PUBLICIDAD SPA | BEFORE PUBLICIDAD,Región Metropolitana de Santiago,Chile,768357935,tenderer,NaN,BEFORE PUBLICIDAD SPA
1,id-0.3.tender.tenderers.1,id-0.3,1093345,MARPRO CHILE SPA. | JAVIERA,id-0.3.parties.9,MARPRO CHILE SPA. | JAVIERA,Región de Valparaíso,Chile,766888429,tenderer,NaN,MARPRO CHILE SPA.
2,id-0.3.tender.tenderers.2,id-0.3,1156776,COMERCIALIZADORA DE ARTÍCULOS DE ASEO JENNIFFE...,id-0.3.parties.2,COMERCIALIZADORA DE ARTÍCULOS DE ASEO JENNIFFE...,Región Metropolitana de Santiago,Chile,770954924,tenderer,NaN,COMERCIALIZADORA DE ARTÍCULOS DE ASEO JENNIFFE...
3,id-0.3.tender.tenderers.3,id-0.3,275627,ETHON PHARMACEUTICALS COMERCIALIZADORA IMP E...,id-0.3.parties.8,ETHON PHARMACEUTICALS COMERCIALIZADORA IMP E...,Región del Libertador General Bernardo O´Higgins,Chile,769561404,tenderer,NaN,ETHON PHARMACEUTICALS COMERCIALIZADORA IMP E...
4,id-0.3.tender.tenderers.4,id-0.3,38669,Crisvert Limitada | Crisvert Limitada,id-0.3.parties.4,Crisvert Limitada | Crisvert Limitada,Región Metropolitana de Santiago,Chile,795914404,tenderer,NaN,VERDEJO Y VERDEJO LIMITADA


In [139]:
df_main_award_itemless = df_main_buyer_info.merge(df_awards, left_on="_link", right_on="_link_main")

In [140]:
df_main_award_itemless.head()

,_link_x,tender_id,initiationType,tender_id,procurement_details,tender_status,procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,buyer_legalName,sector,Codigo Unidad de Compra,Unidad de Compra,_link_y,_link_main,date,status,value_amount,value_monetary_unit
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,SECCIÓN COMPRAS X ZONA,FFAA,1079639,SECCI�N COMPRAS ZONA DE CARABINEROS LOS LAGOS,id-0.4.awards.0,id-0.4,2017-12-21T18:17:23Z,active,6000000.0,CLP
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,COMPLEJO ASISTENCIAL DR. SOTERO DEL RIO,SALUD,1058095,F�RMARCOS,id-0.9.awards.0,id-0.9,2018-01-30T11:15:26Z,unsuccessful,NaN,NaN
2,id-0.10,ocds-70d2nz-4993-132-L121-2021-12-22T17:49:37Z,tender,4993-132-L121,L1,unsuccessful,open,False,2021-12-16 15:41:00,2021-12-15 15:41:00,...,MUNICIPALIDAD DE CHOLCHOL,MUNICIPALIDADES,5906,SECPLAN,id-0.10.awards.0,id-0.10,2017-12-14T15:12:42Z,active,3000000.0,CLP
3,id-0.11,ocds-70d2nz-4019-53-L121-2021-12-14T19:52:56Z,tender,4019-53-L121,L1,complete,open,True,2021-12-13 15:01:00,2021-12-10 15:01:00,...,ILUSTRE MUNICIPALIDAD DE CAMINA,MUNICIPALIDADES,4939,DEPTO. DE FINANZAS,id-0.11.awards.0,id-0.11,2018-01-08T11:20:54Z,active,12605000.0,CLP
4,id-0.14,ocds-70d2nz-780724-18-LE21-2021-12-22T18:09:39Z,tender,780724-18-LE21,LE,complete,open,True,2021-12-23 18:00:00,2021-12-20 15:15:00,...,ILUSTRE TERCER TRIBUNAL AMBIENTAL,LEGISLATIVO Y JUDICIAL,780724,ILUSTRE TERCER TRIBUNAL AMBIENTAL,id-0.14.awards.0,id-0.14,2017-12-22T14:17:12Z,active,890400.0,CLP


In [141]:
df_main_award_itemless.drop(columns=["_link_y", "_link_main"], inplace=True)

In [142]:
df_main_tenderers = df_main_buyer_info.merge(df_tenderers, left_on="_link", right_on = "_link_main", how="left")

In [143]:
df_main_tenderers.head()

,_link_x,tender_id,initiationType,tender_id,procurement_details,tender_status,procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,id,name_x,_link_x,name_y,address_region,address_countryName,identifier_id_y,rol 1,rol 2,legalName
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,1200298.0,VICTOR ALEJANDRO SOTO SOTO | VICTOR ALEJANDRO ...,id-0.4.parties.1,VICTOR ALEJANDRO SOTO SOTO | VICTOR ALEJANDRO ...,Región de los Lagos,Chile,135932566,tenderer,NaN,VICTOR ALEJANDRO SOTO SOTO
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,1021518.0,PHARMAVISIÓN SPA | PHARMAVISION SPA,id-0.9.parties.17,PHARMAVISIÓN SPA | PHARMAVISION SPA,Región Metropolitana de Santiago,Chile,768627134,tenderer,NaN,PHARMAVISIÓN SPA
2,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,106395.0,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,id-0.9.parties.12,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,Región Metropolitana de Santiago,Chile,775969407,supplier,tenderer,LABORATORIO CHILE S A
3,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,24160.0,NOVARTIS CHILE S A | NOVARTIS CHILE S A,id-0.9.parties.15,NOVARTIS CHILE S A | NOVARTIS CHILE S A,Región Metropolitana de Santiago,Chile,83002400K,tenderer,NaN,NOVARTIS CHILE S A
4,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,24938.0,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,id-0.9.parties.13,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,Región del Biobío,Chile,876744007,supplier,tenderer,LABORATORIO PASTEUR S.A.


In [144]:
df_main_tenderers.columns

Index(['_link_x', 'tender_id', 'initiationType', 'tender_id',
       'procurement_details', 'tender_status', 'procurementMethod',
       'tender_hasEnquiries', 'tender_awardPeriod_endDate',
       'tender_awardPeriod_startDate', 'tender_awardPeriod_durationInDays',
       'tender_tenderPeriod_endDate', 'tender_tenderPeriod_startDate',
       'tender_tenderPeriod_durationInDays', 'tender_enquiryPeriod_endDate',
       'tender_enquiryPeriod_startDate', 'tender_enquiryPeriod_durationInDays',
       'estimated_cost', 'buyer_name', 'estimated_cost_monetary_unit',
       'buyer_id', 'buyer_name', 'buyer_region', 'identifier_id_x',
       'buyer_legalName', 'sector', 'Codigo Unidad de Compra',
       'Unidad de Compra', '_link_y', '_link_main', 'id', 'name_x', '_link_x',
       'name_y', 'address_region', 'address_countryName', 'identifier_id_y',
       'rol 1', 'rol 2', 'legalName'],
      dtype='object')

In [145]:
df_main_tenderers["Has_tender"] = df_main_tenderers.index.map(lambda x: 1 if df_main_tenderers.loc[x]["rol 1"] == "tenderer" else 0)
df_main_tenderers["is_supplier"] = df_main_tenderers.index.map(lambda x: 1 if df_main_tenderers.loc[x]["rol 2"] == "supplier" else 0)


In [146]:
df_main_tenderers["identifier_id_y"]

0         135932566
1         768627134
2         775969407
3         83002400K
4         876744007
            ...    
439669    763339807
439670    796761105
439671    788973209
439672    761095560
439673    764199251
Name: identifier_id_y, Length: 439674, dtype: object

In [147]:
df_main_tenderers["identifier_id_x"]

0         619793005
1         616085026
2         616085026
3         616085026
4         616085026
            ...    
439669    616075039
439670    616075039
439671    616075039
439672    616075039
439673    616075039
Name: identifier_id_x, Length: 439674, dtype: object

In [148]:
df_main_tenderers.drop(columns=["name_x"],inplace=True)

In [149]:
df_main_tenderers.rename(columns={"_link_x":"_link", "name_y":"tender_name", "address_region":"tenderer_region", "address_countryName":"tenderer_country",
                                  "legalName":"tenderer_legalName", "identifier_id_y":"tenderer_identifier", "identifier_id_x":"buyer_identifier"},inplace=True)
df_main_tenderers.head()

,_link,tender_id,initiationType,tender_id,procurement_details,tender_status,procurementMethod,tender_hasEnquiries,tender_awardPeriod_endDate,tender_awardPeriod_startDate,...,_link,tender_name,tenderer_region,tenderer_country,tenderer_identifier,rol 1,rol 2,tenderer_legalName,Has_tender,is_supplier
0,id-0.4,ocds-70d2nz-1079639-135-L121-2021-12-27T17:31:16Z,tender,1079639-135-L121,L1,unsuccessful,open,False,2021-12-22 18:00:00,2021-12-07 15:10:00,...,id-0.4.parties.1,VICTOR ALEJANDRO SOTO SOTO | VICTOR ALEJANDRO ...,Región de los Lagos,Chile,135932566,tenderer,NaN,VICTOR ALEJANDRO SOTO SOTO,1,0
1,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,id-0.9.parties.17,PHARMAVISIÓN SPA | PHARMAVISION SPA,Región Metropolitana de Santiago,Chile,768627134,tenderer,NaN,PHARMAVISIÓN SPA,1,0
2,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,id-0.9.parties.12,LABORATORIO CHILE S A | LABORATORIO CHILE S.A.,Región Metropolitana de Santiago,Chile,775969407,supplier,tenderer,LABORATORIO CHILE S A,0,0
3,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,id-0.9.parties.15,NOVARTIS CHILE S A | NOVARTIS CHILE S A,Región Metropolitana de Santiago,Chile,83002400K,tenderer,NaN,NOVARTIS CHILE S A,1,0
4,id-0.9,ocds-70d2nz-1058095-73-LE21-2021-12-17T13:25:56Z,tender,1058095-73-LE21,LE,complete,open,False,2021-12-30 17:29:00,2021-12-09 15:02:00,...,id-0.9.parties.13,LABORATORIO PASTEUR S.A. | Laboratorio Pasteur SA,Región del Biobío,Chile,876744007,supplier,tenderer,LABORATORIO PASTEUR S.A.,0,0


In [150]:
df_main.columns

Index(['_link', 'id', 'initiationType', 'tender_id',
       'tender_procurementMethodDetails', 'tender_status',
       'tender_procurementMethod', 'tender_hasEnquiries',
       'tender_awardPeriod_endDate', 'tender_awardPeriod_startDate',
       'tender_awardPeriod_durationInDays', 'tender_tenderPeriod_endDate',
       'tender_tenderPeriod_startDate', 'tender_tenderPeriod_durationInDays',
       'tender_enquiryPeriod_endDate', 'tender_enquiryPeriod_startDate',
       'tender_enquiryPeriod_durationInDays', 'tender_procuringEntity_name',
       'tender_value_amount', 'tender_value_unitOfAccount',
       'tender_value_currency', 'buyer_id', 'buyer_name'],
      dtype='object')

# Datasets del 2021
Aquí creamos con los siguientes comandos (puestos en comentario por si acaso) los datasets de awards y tenders de 2021

In [151]:
df_main_tenderers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439674 entries, 0 to 439673
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype          
---  ------                               --------------   -----          
 0   _link                                439674 non-null  object         
 1   tender_id                            439674 non-null  object         
 2   initiationType                       439674 non-null  object         
 3   tender_id                            439674 non-null  object         
 4   procurement_details                  438259 non-null  category       
 5   tender_status                        439674 non-null  object         
 6   procurementMethod                    439674 non-null  object         
 7   tender_hasEnquiries                  439674 non-null  bool           
 8   tender_awardPeriod_endDate           439674 non-null  datetime64[ns] 
 9   tender_awardPeriod_startDate         439674 non-null  datet

In [152]:
df_main_award.to_excel('awards_2021.xlsx', index=False)

In [153]:
df_main_tenderers.to_excel('tenderers_2021.xlsx', index=False)